# Sample California Experiment Readout Code: [CA Discount Factor Experiment](https://docs.google.com/document/d/1Dq9t_Lwx8DxirVgumsbNwRlHWHZ6KMSSY4OueSyWEcY/edit#)

**TL;DR:** In this notebook, the readout analysis for the cost and non-cost metrics in the CA Discount Factor Experiment are summarized. The same approach can be used for other CA experiments where the [CA randomization scheme](https://instacart.atlassian.net/wiki/spaces/Fulfillment/pages/3359474508/The+Day-Of-Week+Enforced+Randomization+in+the+Logistics+Experimentation+Platform#Notes-for-California-(CA)-Experiments) is implemented. 

**POC:** Ozge Islegen

In [80]:
# Import packages for data visualization and statistical analysis

###########################################
# Suppress matplotlib user warnings
# Necessary for newer version of matplotlib
import warnings
warnings.filterwarnings("ignore")
#
# Display inline matplotlib plots with IPython
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
###########################################

# Import libraries for data visualization
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patches as mpatches
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set()

# Pretty display for notebooks
%matplotlib inline

# Optional global parameter tuning for data visualization 
from pylab import rcParams
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
matplotlib.rcParams['axes.titlesize']=16
rcParams['figure.figsize'] = 14, 9

# For datetime operations
import datetime
from datetime import timedelta
from datetime import date

# Import packages for data analysis

# Check the versions of key python libraries
# scipy
import scipy
print('scipy: %s' % scipy.__version__)
# numpy
import numpy as np
print('numpy: %s' % np.__version__)

# pandas
import pandas as pd
print('pandas: %s' % pd.__version__)
# Package for statistical models
import statsmodels.api as sm
import statsmodels
print('statsmodels: %s' % statsmodels.__version__)
# scikit-learn
import sklearn
print('sklearn: %s' % sklearn.__version__)


from builtins import object
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import itertools

import math # for mathematical functions such as logarithm
# Package for fast EDA
import pandas_profiling

# For running sql queries in Snowflake
import instaquery as iq

scipy: 1.4.1
numpy: 1.21.0
pandas: 1.2.5
statsmodels: 0.11.1
sklearn: 0.22.2.post1


# 1. Overpay, Guaranteed Pay and Non-Cost Metric Definitions 

## 1.1 Overpay Calculations

Overpay metric for a given shopper-week is defined as:

Overpay= total pay - guaranteed pay

where total pay= upfront pay + weekly adjustment.

**1. Assigning each shopper-day to a single variant:**

We use the simulation data at the shopper-day level and assign each shopper-day to the variant where the shopper spent the most active hours that day. 
Example: Suppose the shopper worked in Zones A, B and C on a given day. The shopper spent 1, 2 and 3 active hours, respectively, delivering to these zones. Zone A is in Control and Zones B and C are in treatment on that day. That means the shopper spent 1 hour in a Control zone and 5 hours in Treatment zones on that day. Therefore, this shopper-day is assigned to the variant Treatment. 

**2. Assigning the Overpay to Zone-Variants** 


a. Use the variant assigned to each shopper-day to calculate, the overpay for each shopper-variant (final_tool_simulation_tool_2 CTE below). \
b. Calculate the total active hours for each zone-shopper-variant using the variant that is assigned to each zone-shopper-date above (zone_stats CTE below).\
c. Calculate the total active hours for each shopper-variant using the variant that is assigned to each zone-shopper-date above (total_stats CTE below).\
d. For each shopper-zone-variant, assign the portion of the overpay (calculated in a.) by the active hours worked by the shopper in that zone-variant (calculated as b./c. above) (zone_driver_variant CTE below).\
e. For each zone-variant, calculate the sum of overpay for each zone-shopper-variant and divide it by the total  number of deliveries


#### Example: 

Suppose the shopper worked in Zones A, B and C on a Day 1 of the week. The shopper spent 1, 2 and 3 active hours, respectively, delivering to these zones. Zone A is in Control and Zones B and C are in treatment that day. That means the shopper spent 1 hour in a Control zone and 5 hours in Treatment zones on that day. Therefore, this shopper-day is assigned to the variant Treatment.

Suppose the same shopper worked in Zones B and C on Day 2 of the week. The shopper spent 3 and 2 active hours, respectively, delivering to these zones. Zone B is in treatment and C is in control on that day. That means the shopper spent 2 hour in a Control zone and 3 hours in Treatment zones on that day. Therefore, this shopper-day is assigned to the variant Treatment.

These data are summarized below:


|     Zones      | A                 | B               |     C          | 
| :------------: | :---------------: | :-------------: | :------------: | 
| Variant-Day 1  |Control            |     Treatment   | Treatment      |        
| Active Hours-Day 1|      1            |       2         |   3            |
| Variant-Day 2  |-           |     Treatment   | Control     |        
| Active Hours-Day 2   |      0            |       3         |   2         |


a. Including all batches on Day 1 and 2 completed by the shopper, calculate the overpay for a given shopper-treatment: \$2. \
b. All zones on day 1 and 2 are assigned to the treatment:

|     Zones      | A                 | B               |     C          | 
| :------------: | :---------------: | :-------------: | :------------: | 
| Variant-Day 1  |Treatment         |     Treatment   | Treatment      |        
| Active Hours-Day 1|      1            |       2         |   3            |
| Variant-Day 2  | Treatment          |     Treatment   | Treatment    |        
| Active Hours-Day 2   |      0            |       3         |   2         |
| Active Hours for Shopper-Treament   |      1            |       5        |   5         |

There is only one shopper-variant=shopper-treatment. As shown in the above table, the total active hours for shopper-treatment-zone A= 1, and so on.

c. The total active hours for the shopper-treatment is 11 hours. \
d. Shopper-zone A-treatment overpay = $\$2 * 1/11= \$2/11$ \
Shopper-zone B-treatment overpay = $\$2 * 5/11= \$10/11$ \
Shopper-zone C-treatment overpay = $\$2 * 5/11= \$10/11$ \
e. Sum up all shopper-zone A-treatment over all shoppers, to get the overpay for Zone A-treatment overpay and so on.

The other details of the logic of the queries are explained in the comments below. Note that we calculate the overpay for each X week cluster where X=# of variants in the experiment. Then, we add the cluster number as a region-cluster covariate into the regression below:

In [81]:
# Write the overpay query for the first three-week block of the experiment using the shopper-day level simulation data
overpay_query_1=""" 
--2016 rows: 96 zones * 21 days
--672 rows for each of the 3 variants
WITH experiment_schedule as (
    select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-02-27' as date)
    ORDER BY 1)

-- Simulation output ""...cents" columns are daily totals of all listed batches in the batch_ids column of the given shopper-day.
   , simulation_output AS (
    select driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents
           --         
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation WHERE experiment_id='580')

   , sim_output_batches AS (
    select DISTINCT driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents,
           f.value    as batch_id
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation
       , table (flatten(input => parse_json(batch_ids))) f WHERE experiment_id='580')

-- One shopper day can be associated with multiple zones and therefore, multiple variants 
   , shopper_zone_day AS (
    SELECT s.date   as date,
           s.driver_id,
         --  rs.shopper_level,
           es.variant,
           ps.zone_id,
           z.name   as zone_name,
           z.state  as state,
           r.id     as region_id,
           r.name   as region_name,
           c.name   as country,
   
           SUM(ZEROIFNULL(fb.n_del)) as del_number,
           sum(case
                   when ps.batch_type = 'drive_and_pick' then
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_started_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
                   else
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_completed_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
               end) AS active_hours
           --  , SUM(s.batch_payment_amount_cents)/100 as zone_day_upfront_pay
           --  ,SUM(s.guaranteed_minimum_amount_cents)/100 as zone_day_guarantee_pay
    FROM rds_data.picking_sets ps
             JOIN rds_data.zones z ON (z.id = ps.zone_id)
             JOIN rds_data.regions r ON r.id = z.region_id
             JOIN rds_data.countries c ON c.id = r.country_id
             JOIN (select batch_id, sum(num_deliveries) as n_del from dwh.fact_wloc_batch fwb WHERE SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y' group by 1) fb ON fb.batch_id=ps.id
             JOIN sim_output_batches s ON s.batch_id = ps.id
             JOIN experiment_schedule es ON es.zone_id = ps.zone_id AND es.date = s.date
           --  LEFT JOIN (select distinct batch_date,
           --                             shopper_id,
           --                             iff(running_total_batches < 11, 'very new', level) as shopper_level
           --             from analysts.shopper_retention_engagement) rs on rs.shopper_id = ps.driver_id and
           --                                                               rs.batch_date =
           --                                                               convert_timezone(z.time_zone_name,
                                                                                           --ps.delivery_completed_at)::date
    WHERE 1 = 1
          --     AND ps.workflow_state = 'completed'
          --  AND fb.SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y'
          -- AND ps.batch_type IN ('delivery', 'rx_delivery', 'drive_and_pick', 'delivery_only', 'pickup_delivery')
          --AND CONVERT_TIMEZONE(z.time_zone_name, ps.delivery_completed_at)::DATE between cast('2021-07-26' as date) and cast('2021-11-14' as date)
          -- AND ps.zone_id in (select zone_id from asp_eligible_zone_list)
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9)

   -- While a shopper can work in multiple zones in a given day, these zones are highly likely to belong to the same variant due to the region-day randomization.
   , shopper_variant_day as (
    SELECT date, driver_id, variant, SUM(active_hours) as active_hours
    FROM shopper_zone_day
    GROUP BY 1, 2, 3)

 -- Assign the variant to the shopper-day that has the maximum active hours for that shopper-day  
   , final2 as (
    SELECT s1.date,
           s1.driver_id,
           s1.variant
           --, COUNT(DISTINCT zone_id), COUNT(DISTINCT region_id), LISTAGG(DISTINCT region_id, ', ') as regions
    FROM shopper_variant_day s1
    WHERE active_hours = (SELECT max(s2.active_hours)
                          FROM shopper_variant_day s2
                          WHERE s2.date = s1.date AND s2.driver_id = s1.driver_id))
-- For a given variant and driver_id, calculate the total pay for the variant for a given week
   , final_tool_simulation_2 as (
    SELECT f2.variant,
           f2.driver_id,
           GREATEST(SUM(daily_adjustment_amount_cents), 0) / 100 as daily_adjustment,
           SUM(batch_payment_amount_cents) / 100                 as total_upfront_pay,
           SUM(guaranteed_minimum_amount_cents) / 100            as total_guarantee_pay,
          daily_adjustment + total_upfront_pay - total_guarantee_pay  as over_pay
    FROM simulation_output s2
    JOIN final2 f2 ON s2.date = f2.date AND s2.driver_id = f2.driver_id
    GROUP BY 1, 2)
-- For a given driver_id, zone and variant (which comes from the variant the shopper spent the most active hours assigned in final2)
   , zone_stats as (
    SELECT
           s.driver_id
         , s.zone_id
         , f.variant
         , SUM(active_hours) as zone_ah
         , SUM(del_number) as zone_del
    FROM shopper_zone_day s
    JOIN final2 f ON s.date = f.date AND s.driver_id = f.driver_id
    GROUP BY 1, 2, 3)
-- For a driver, variant, what is the total active hours?
   , total_stats as (
       SELECT
              s.driver_id
            , f.variant
            , SUM(active_hours) as total_ah
            --, SUM(del_number) total_del
            FROM shopper_zone_day s
                   JOIN final2 f ON s.date = f.date AND s.driver_id = f.driver_id
            GROUP BY 1, 2)

   , zone_driver_variant as (
    SELECT z.driver_id,
           z.zone_id,
           zo.region_id,
           z.variant,
           zone_del,
           zone_ah,
           total_ah,
           over_pay * zone_ah / total_ah as zone_driver_variant_over_pay
    FROM zone_stats z
             JOIN final_tool_simulation_2 f ON z.driver_id = f.driver_id AND z.variant = f.variant
             JOIN total_stats t ON z.variant = t.variant AND z.driver_id = t.driver_id
             JOIN rds_data.zones zo ON z.zone_id=zo.id 
   WHERE total_ah>0)

    /*
    , global_orders_table as (SELECT
      COUNT(ORDER_DELIVERY_ID)         as    global_orders
    FROM dwh.FACT_ORDER_DELIVERY FOD
     JOIN rds_data.zones z ON fod.zone_id=z.id
    WHERE TRUE
       AND z.active=True
      AND DELIVERY_STATE = 'delivered'
      AND CONVERT_TIMEZONE('UTC', z.time_zone_name, delivered_date_time_utc)::DATE
                     BETWEEN CAST('2022-01-03' AS DATE) AND CAST('2022-01-23' AS DATE)
                    )

    */
   , zone_variant as (
    SELECT zone_id, region_id, variant,SUM(zone_driver_variant_over_pay) / SUM(zone_del) as overpay_per_del, SUM(zone_del) as total_deliveries
    FROM zone_driver_variant
    GROUP BY 1,2,3)

SELECT * --COUNT(DISTINCT zone_id), COUNT(DISTINCT driver_id), COUNT(DISTINCT variant), max(zone_ah),min(zone_ah), avg(zone_ah), max(zone_del), min(zone_del), avg(zone_del)
FROM zone_variant
--WHERE zone_del > 200
--LIMIT 100
"""

In [82]:
# Write the overpay query for the second three-week block of the experiment using the shopper-day level simulation data
overpay_query_2=""" --2016 rows: 96 zones * 21 days
--672 rows for each of the 3 variants
WITH experiment_schedule as (
    select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-28' as date) and cast('2022-03-20' as date)
    ORDER BY 1)

-- Simulation output ""...cents" columns are daily totals of all listed batches in the batch_ids column of the given shopper-day.
   , simulation_output AS (
    select driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents
           --         
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation WHERE experiment_id='580')

   , sim_output_batches AS (
    select DISTINCT driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents,
           f.value    as batch_id
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation
       , table (flatten(input => parse_json(batch_ids))) f WHERE experiment_id='580')

-- One shopper day can be associated with multiple zones and therefore, multiple variants 
   , shopper_zone_day AS (
    SELECT s.date   as date,
           s.driver_id,
         --  rs.shopper_level,
           es.variant,
           ps.zone_id,
           z.name   as zone_name,
           z.state  as state,
           r.id     as region_id,
           r.name   as region_name,
           c.name   as country,
   
           SUM(ZEROIFNULL(fb.n_del)) as del_number,
           sum(case
                   when ps.batch_type = 'drive_and_pick' then
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_started_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
                   else
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_completed_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
               end) AS active_hours
           --  , SUM(s.batch_payment_amount_cents)/100 as zone_day_upfront_pay
           --  ,SUM(s.guaranteed_minimum_amount_cents)/100 as zone_day_guarantee_pay
    FROM rds_data.picking_sets ps
             JOIN rds_data.zones z ON (z.id = ps.zone_id)
             JOIN rds_data.regions r ON r.id = z.region_id
             JOIN rds_data.countries c ON c.id = r.country_id
             JOIN (select batch_id, sum(num_deliveries) as n_del from dwh.fact_wloc_batch fwb WHERE SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y' group by 1) fb ON fb.batch_id=ps.id
             JOIN sim_output_batches s ON s.batch_id = ps.id
             JOIN experiment_schedule es ON es.zone_id = ps.zone_id AND es.date = s.date
           --  LEFT JOIN (select distinct batch_date,
           --                             shopper_id,
           --                             iff(running_total_batches < 11, 'very new', level) as shopper_level
           --             from analysts.shopper_retention_engagement) rs on rs.shopper_id = ps.driver_id and
           --                                                               rs.batch_date =
           --                                                               convert_timezone(z.time_zone_name,
                                                                                           --ps.delivery_completed_at)::date
    WHERE 1 = 1
          --     AND ps.workflow_state = 'completed'
          --  AND fb.SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y'
          -- AND ps.batch_type IN ('delivery', 'rx_delivery', 'drive_and_pick', 'delivery_only', 'pickup_delivery')
          --AND CONVERT_TIMEZONE(z.time_zone_name, ps.delivery_completed_at)::DATE between cast('2021-07-26' as date) and cast('2021-11-14' as date)
          -- AND ps.zone_id in (select zone_id from asp_eligible_zone_list)
    GROUP BY 1, 2, 3, 4, 5, 6, 7, 8, 9)

   -- While a shopper can work in multiple zones in a given day, these zones are highly likely to belong to the same variant due to the region-day randomization.
   , shopper_variant_day as (
    SELECT date, driver_id, variant, SUM(active_hours) as active_hours
    FROM shopper_zone_day
    GROUP BY 1, 2, 3)


   , final2 as (
    SELECT s1.date,
           s1.driver_id,
           s1.variant
           --, COUNT(DISTINCT zone_id), COUNT(DISTINCT region_id), LISTAGG(DISTINCT region_id, ', ') as regions
    FROM shopper_variant_day s1
    WHERE active_hours = (SELECT max(s2.active_hours)
                          FROM shopper_variant_day s2
                          WHERE s2.date = s1.date AND s2.driver_id = s1.driver_id))
-- For a given variant and driver_id, calculate the total pay for the variant
   , final_tool_simulation_2 as (
    SELECT f2.variant,
           f2.driver_id,
           GREATEST(SUM(daily_adjustment_amount_cents), 0) / 100 as daily_adjustment,
           SUM(batch_payment_amount_cents) / 100                 as total_upfront_pay,
           SUM(guaranteed_minimum_amount_cents) / 100            as total_guarantee_pay,
          daily_adjustment + total_upfront_pay - total_guarantee_pay  as over_pay
    FROM simulation_output s2
    JOIN final2 f2 ON s2.date = f2.date AND s2.driver_id = f2.driver_id
    GROUP BY 1, 2)
-- For a given driver_id, zone and variant (which comes from the variant the shopper spent the most active hours assigned in final2)
   , zone_stats as (
    SELECT
           s.driver_id
         , s.zone_id
         , f.variant
         , SUM(active_hours) as zone_ah
         , SUM(del_number) as zone_del
    FROM shopper_zone_day s
    JOIN final2 f ON s.date = f.date AND s.driver_id = f.driver_id
    GROUP BY 1, 2, 3)
-- For a driver, variant, what is the total active hours?
   , total_stats as (
       SELECT
              s.driver_id
            , f.variant
            , SUM(active_hours) as total_ah
            --, SUM(del_number) total_del
            FROM shopper_zone_day s
                   JOIN final2 f ON s.date = f.date AND s.driver_id = f.driver_id
            GROUP BY 1, 2)

   , zone_driver_variant as (
    SELECT z.driver_id,
           z.zone_id,
           zo.region_id,
           z.variant,
           zone_del,
           zone_ah,
           total_ah,
           over_pay * zone_ah / total_ah as zone_driver_variant_over_pay
    FROM zone_stats z
             JOIN final_tool_simulation_2 f ON z.driver_id = f.driver_id AND z.variant = f.variant
             JOIN total_stats t ON z.variant = t.variant AND z.driver_id = t.driver_id
             JOIN rds_data.zones zo ON z.zone_id=zo.id 
   WHERE total_ah>0)

    /*
    , global_orders_table as (SELECT
      COUNT(ORDER_DELIVERY_ID)         as    global_orders
    FROM dwh.FACT_ORDER_DELIVERY FOD
     JOIN rds_data.zones z ON fod.zone_id=z.id
    WHERE TRUE
       AND z.active=True
      AND DELIVERY_STATE = 'delivered'
      AND CONVERT_TIMEZONE('UTC', z.time_zone_name, delivered_date_time_utc)::DATE
                     BETWEEN CAST('2022-01-03' AS DATE) AND CAST('2022-01-23' AS DATE)
                    )

    */
   , zone_variant as (
    SELECT zone_id, region_id, variant,SUM(zone_driver_variant_over_pay) / SUM(zone_del) as overpay_per_del, SUM(zone_del) as total_deliveries
    FROM zone_driver_variant
    GROUP BY 1,2,3)

SELECT * --COUNT(DISTINCT zone_id), COUNT(DISTINCT driver_id), COUNT(DISTINCT variant), max(zone_ah),min(zone_ah), avg(zone_ah), max(zone_del), min(zone_del), avg(zone_del)
FROM zone_variant
--WHERE zone_del > 200
--LIMIT 100
"""

## 1.2 Guaranteed Pay Calculations

In this subsection, we calculate the guaranteed pay (or [earnings guarantee](https://instacart.atlassian.net/wiki/spaces/Fulfillment/pages/3175581541/Batch+Pay+Documentation#*Important-Note%3A-California-Components-Differ-Due-to-Prop-22*)) per delivery (the amount of pay promised for the actual active time and mileage under Proposition 22 in California).

The guaranteed pay per delivery for each zone-variant is calculated exactly by using the batch-level actual time and mileage data that is available in rds_data.weekly_adjustments as opposed to the overpay calculations above that divides the overpay across zone-variants approximately.

The other details of the logic of the queries are explained in the comments below. Note that we calculate the overpay for each X week cluster where X=# of variants in the experiment. Then, we add the cluster number as a region-cluster covariate into the regression below:

In [83]:
guaranteed_pay_query_1="""
--2016 rows: 96 zones * 21 days
--672 rows for each of the 3 variants
WITH experiment_schedule as (
    select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-02-27' as date)
    ORDER BY 1)

-- Simulation output ""...cents" columns are daily totals of all listed batches in the batch_ids column of the given shopper-day.
   , simulation_output AS (
    select driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents
           --         
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation WHERE experiment_id='580')

   , sim_output_batches AS (
    select DISTINCT driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents,
           f.value    as batch_id
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation
       , table (flatten(input => parse_json(batch_ids))) f WHERE experiment_id='580')
       
       ,weekly_adjustment_data as (
      select
          wa.*
          ,parse_json(data):batches x
      from rds_data.weekly_adjustments wa
      LEFT JOIN INSTADATA.RDS_DATA.PAY_PERIODS pp ON wa.PAY_PERIOD_ID = pp.ID
      INNER JOIN
        ( SELECT driver_id, PAY_PERIOD_ID, MAX(UPDATED_AT) UPDATED_AT
          FROM rds_data.weekly_adjustments
          GROUP BY 1,2
        ) dedupe_key
        ON wa.DRIVER_ID = dedupe_key.DRIVER_ID
        AND wa.PAY_PERIOD_ID = dedupe_key.PAY_PERIOD_ID
        AND wa.UPDATED_AT = dedupe_key.UPDATED_AT

      WHERE TRUE
        AND pp.STARTS_AT::DATE >= '2022-02-07'
)

, wa_batch_counted AS (
    SELECT
        value:batch_id as batch_id,
        value:distance_miles_ceiling as batch_miles_EG,
        value:duration_minutes_ceiling as batch_mins_EG,
        value:min_wage_rate_amount_cents / 100 as min_wage,
        1.2 * min_wage * (batch_mins_EG / 60) as batch_time_EG_pay,
        batch_miles_EG * 0.3 as batch_distance_EG_pay,
        batch_time_EG_pay+batch_distance_EG_pay as eg_pay,
        ANY_VALUE(driver_id) as driver_id,
        ANY_VALUE(pay_period_id) as pay_period_id,
        ANY_VALUE(guaranteed_minimum_amount_cents / 100) as guarantee_pay
    FROM weekly_adjustment_data
        ,lateral flatten(input => x)
    GROUP BY 1,2,3,4
    ORDER BY 4 desc
)       

   , zone_variant AS (
    SELECT --s.date   as date,
           --s.driver_id,
         --  rs.shopper_level,
           es.variant,
           ps.zone_id,
           z.name   as zone_name,
           z.state  as state,
           r.id     as region_id,
           r.name   as region_name,
           c.name   as country,
        SUM(eg_pay) as total_eg_pay, 
   
           SUM(ZEROIFNULL(fb.n_del)) as del_number,
        total_eg_pay/del_number as guaranteed_pay_per_del,
           sum(case
                   when ps.batch_type = 'drive_and_pick' then
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_started_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
                   else
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_completed_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
               end) AS active_hours
           --  , SUM(s.batch_payment_amount_cents)/100 as zone_day_upfront_pay
           --  ,SUM(s.guaranteed_minimum_amount_cents)/100 as zone_day_guarantee_pay
    FROM rds_data.picking_sets ps
             JOIN rds_data.zones z ON (z.id = ps.zone_id)
             JOIN rds_data.regions r ON r.id = z.region_id
             JOIN rds_data.countries c ON c.id = r.country_id
             JOIN (select batch_id, sum(num_deliveries) as n_del from dwh.fact_wloc_batch fwb WHERE SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y' group by 1) fb ON fb.batch_id=ps.id
             JOIN sim_output_batches s ON s.batch_id = ps.id
             JOIN  wa_batch_counted w ON w.batch_id = s.batch_id
             JOIN experiment_schedule es ON es.zone_id = ps.zone_id AND es.date = s.date
           --  LEFT JOIN (select distinct batch_date,
           --                             shopper_id,
           --                             iff(running_total_batches < 11, 'very new', level) as shopper_level
           --             from analysts.shopper_retention_engagement) rs on rs.shopper_id = ps.driver_id and
           --                                                               rs.batch_date =
           --                                                               convert_timezone(z.time_zone_name,
                                                                                           --ps.delivery_completed_at)::date
    WHERE 1 = 1
          --     AND ps.workflow_state = 'completed'
          --  AND fb.SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y'
          -- AND ps.batch_type IN ('delivery', 'rx_delivery', 'drive_and_pick', 'delivery_only', 'pickup_delivery')
          --AND CONVERT_TIMEZONE(z.time_zone_name, ps.delivery_completed_at)::DATE between cast('2021-07-26' as date) and cast('2021-11-14' as date)
          -- AND ps.zone_id in (select zone_id from asp_eligible_zone_list)
    GROUP BY 1, 2, 3, 4, 5, 6, 7)
    
    
    SELECT zone_id, variant, guaranteed_pay_per_del
    FROM zone_variant
"""

In [84]:
guaranteed_pay_query_2="""
--2016 rows: 96 zones * 21 days
--672 rows for each of the 3 variants
WITH experiment_schedule as (
    select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-28' as date) and cast('2022-03-20' as date)
    ORDER BY 1)

-- Simulation output ""...cents" columns are daily totals of all listed batches in the batch_ids column of the given shopper-day.
   , simulation_output AS (
    select driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents
           --         
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation WHERE experiment_id='580')

   , sim_output_batches AS (
    select DISTINCT driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents,
           f.value    as batch_id
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation
       , table (flatten(input => parse_json(batch_ids))) f WHERE experiment_id='580')
       
       ,weekly_adjustment_data as (
      select
          wa.*
          ,parse_json(data):batches x
      from rds_data.weekly_adjustments wa
      LEFT JOIN INSTADATA.RDS_DATA.PAY_PERIODS pp ON wa.PAY_PERIOD_ID = pp.ID
      INNER JOIN
        ( SELECT driver_id, PAY_PERIOD_ID, MAX(UPDATED_AT) UPDATED_AT
          FROM rds_data.weekly_adjustments
          GROUP BY 1,2
        ) dedupe_key
        ON wa.DRIVER_ID = dedupe_key.DRIVER_ID
        AND wa.PAY_PERIOD_ID = dedupe_key.PAY_PERIOD_ID
        AND wa.UPDATED_AT = dedupe_key.UPDATED_AT

      WHERE TRUE
        AND pp.STARTS_AT::DATE >= '2022-02-07'
)

, wa_batch_counted AS (
    SELECT
        value:batch_id as batch_id,
        value:distance_miles_ceiling as batch_miles_EG,
        value:duration_minutes_ceiling as batch_mins_EG,
        value:min_wage_rate_amount_cents / 100 as min_wage,
        1.2 * min_wage * (batch_mins_EG / 60) as batch_time_EG_pay,
        batch_miles_EG * 0.3 as batch_distance_EG_pay,
        batch_time_EG_pay+batch_distance_EG_pay as eg_pay,
        ANY_VALUE(driver_id) as driver_id,
        ANY_VALUE(pay_period_id) as pay_period_id,
        ANY_VALUE(guaranteed_minimum_amount_cents / 100) as guarantee_pay
    FROM weekly_adjustment_data
        ,lateral flatten(input => x)
    GROUP BY 1,2,3,4
    ORDER BY 4 desc
)       

   , zone_variant AS (
    SELECT --s.date   as date,
           --s.driver_id,
         --  rs.shopper_level,
           es.variant,
           ps.zone_id,
           z.name   as zone_name,
           z.state  as state,
           r.id     as region_id,
           r.name   as region_name,
           c.name   as country,
        SUM(eg_pay) as total_eg_pay, 
   
           SUM(ZEROIFNULL(fb.n_del)) as del_number,
        total_eg_pay/del_number as guaranteed_pay_per_del,
           sum(case
                   when ps.batch_type = 'drive_and_pick' then
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_started_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
                   else
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_completed_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
               end) AS active_hours
           --  , SUM(s.batch_payment_amount_cents)/100 as zone_day_upfront_pay
           --  ,SUM(s.guaranteed_minimum_amount_cents)/100 as zone_day_guarantee_pay
    FROM rds_data.picking_sets ps
             JOIN rds_data.zones z ON (z.id = ps.zone_id)
             JOIN rds_data.regions r ON r.id = z.region_id
             JOIN rds_data.countries c ON c.id = r.country_id
             JOIN (select batch_id, sum(num_deliveries) as n_del from dwh.fact_wloc_batch fwb WHERE SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y' group by 1) fb ON fb.batch_id=ps.id
             JOIN sim_output_batches s ON s.batch_id = ps.id
             JOIN  wa_batch_counted w ON w.batch_id = s.batch_id
             JOIN experiment_schedule es ON es.zone_id = ps.zone_id AND es.date = s.date
           --  LEFT JOIN (select distinct batch_date,
           --                             shopper_id,
           --                             iff(running_total_batches < 11, 'very new', level) as shopper_level
           --             from analysts.shopper_retention_engagement) rs on rs.shopper_id = ps.driver_id and
           --                                                               rs.batch_date =
           --                                                               convert_timezone(z.time_zone_name,
                                                                                           --ps.delivery_completed_at)::date
    WHERE 1 = 1
          --     AND ps.workflow_state = 'completed'
          -- AND fb.SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y'
          -- AND ps.batch_type IN ('delivery', 'rx_delivery', 'drive_and_pick', 'delivery_only', 'pickup_delivery')
          -- AND CONVERT_TIMEZONE(z.time_zone_name, ps.delivery_completed_at)::DATE between cast('2021-07-26' as date) and cast('2021-11-14' as date)
          -- AND ps.zone_id in (select zone_id from asp_eligible_zone_list)
    GROUP BY 1, 2, 3, 4, 5, 6, 7)
    
    
    SELECT zone_id, variant, guaranteed_pay_per_del
    FROM zone_variant
"""

## 1.3 The Non-Cost Metrics Calculations

The non-cost metrics are zone-day metrics directly taken from the [logistics experimentation platfrom](https://github.com/instacart/data-eng-services/tree/master/etl/fulfillment/models/zone_day) and calculated for each zone-date-variant.

In [85]:
noncost_query="""with zone_dates as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-03-20' as date)
    ORDER BY 1
    )

, deliveries_late as (
  SELECT
  od.id as delivery_id,
  od.zone_id,
  od.delivery_type,
  od.workflow_state,
  od.created_at,
  od.window_starts_at,
  od.window_ends_at,
  case when od.delivery_type = 'pickup' then 1 else 0 end as is_pickup,
  ps.batch_type,
  DATE_TRUNC('day', CONVERT_TIMEZONE(z.time_zone_name, od.delivered_at))::DATE date,
  case when od.delivery_type = 'pickup' then ps.delivery_started_at else od.delivered_at end as delivered_at,
  case when od.delivery_type = 'pickup' then ps.planned_delivery_started_at else ps.PLANNED_DELIVERY_COMPLETED_AT end as planned_delivered_at,
  case when od.delivery_type = 'pickup' then od.window_starts_at else od.window_ends_at end as due_at,
  case when (od.delivery_type = 'pickup') then datediff('minute', od.window_starts_at, ps.delivery_started_at)
       else datediff('minute', od.window_ends_at, od.delivered_at) end as lateness_minutes
  FROM rds_data.order_deliveries od
  INNER JOIN rds_data.zones z ON od.zone_id = z.id
  INNER JOIN rds_data.order_delivery_batches odb ON od.id = odb.order_delivery_id
  INNER JOIN rds_data.picking_sets ps on ps.id = odb.batch_id
JOIN zone_dates zd on zd.zone_id = od.zone_id and zd.date=CONVERT_TIMEZONE('UTC', z.time_zone_name, od.delivered_at)::DATE
  WHERE od.workflow_state = 'delivered'
  AND ps.workflow_state = 'completed'
  AND odb.canceled_at IS NULL
  AND ps.batch_type not in ('delivery_only', 'runner_customer')
)
, lateness_metrics as (
  select
    date
    , zone_id
    , avg(lateness_minutes) as avg_lateness_minutes
    , PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY lateness_minutes) as median_lateness_minutes
    , PERCENTILE_CONT(0.80) WITHIN GROUP (ORDER BY lateness_minutes) as p80_lateness_minutes  -- 20% late target
    , PERCENTILE_CONT(0.85) WITHIN GROUP (ORDER BY lateness_minutes) as p85_lateness_minutes  -- 15% late target
    , PERCENTILE_CONT(0.90) WITHIN GROUP (ORDER BY lateness_minutes) as p90_lateness_minutes  -- 10% late target
    , PERCENTILE_CONT(0.95) WITHIN GROUP (ORDER BY lateness_minutes) as p95_lateness_minutes
    , PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY lateness_minutes) as p99_lateness_minutes
  from deliveries_late
  where 1=1
  and delivered_at::date = due_at::date
  and workflow_state = 'delivered'
  group by 1, 2
)

, data_late as (
SELECT
  d.date as date,
  d.zone_id,
  COALESCE(avg_lateness_minutes, 0) as avg_lateness_minutes,
  COALESCE(median_lateness_minutes, 0) as median_lateness_minutes,
  COALESCE(p80_lateness_minutes, 0) as p80_lateness_minutes,
  COALESCE(p85_lateness_minutes, 0) as p85_lateness_minutes,
  COALESCE(p90_lateness_minutes, 0) as p90_lateness_minutes,
  COALESCE(p95_lateness_minutes, 0) as p95_lateness_minutes,
  COALESCE(p99_lateness_minutes, 0) as p99_lateness_minutes,

  count(distinct delivery_id)
    AS deliveries_count,
  count(distinct CASE WHEN delivery_type != 'pickup' THEN delivery_id ELSE NULL END)
    AS delivery_deliveries_count,
  count(distinct CASE WHEN delivery_type = 'pickup' THEN delivery_id ELSE NULL END)
    AS pickup_deliveries_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND batch_type IN ('drive_and_pick', 'pickup_delivery') THEN delivery_id ELSE NULL END)
    AS fss_pickup_deliveries_count,  
  count(distinct CASE WHEN delivery_type = 'pickup' AND batch_type IN ('pickup') THEN delivery_id ELSE NULL END)
    AS partner_pickup_deliveries_count,    
  count(distinct CASE WHEN delivery_type = 'eta' THEN delivery_id ELSE NULL END)
    AS standard_eta_deliveries_count,
  count(distinct CASE WHEN delivery_type = 'priority_eta' THEN delivery_id ELSE NULL END)
    AS priority_eta_deliveries_count,
  count(distinct CASE WHEN delivery_type IN ('scheduled', 'limited_availability') THEN delivery_id ELSE NULL END)
    AS scheduled_deliveries_count,

  count(distinct CASE WHEN delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS late_count,
  count(distinct CASE WHEN delivery_type != 'pickup' AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS delivery_late_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS pickup_late_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND batch_type IN ('drive_and_pick', 'pickup_delivery') AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS fss_pickup_late_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND batch_type IN ('pickup') AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS partner_pickup_late_count,    
  count(distinct CASE WHEN delivery_type = 'eta' AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS standard_eta_late_count,
  count(distinct CASE WHEN delivery_type = 'priority_eta' AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS priority_eta_late_count,
  count(distinct CASE WHEN delivery_type IN ('scheduled', 'limited_availability') AND delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS scheduled_late_count,

  count(distinct CASE WHEN delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS late_15min_count,
  count(distinct CASE WHEN delivery_type != 'pickup' AND delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS delivery_late_15min_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS pickup_late_15min_count,
  count(distinct CASE WHEN delivery_type = 'eta' AND delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS standard_eta_late_15min_count,
  count(distinct CASE WHEN delivery_type = 'priority_eta' AND delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS priority_eta_late_15min_count,
  count(distinct CASE WHEN delivery_type IN ('scheduled', 'limited_availability') AND delivered_at > dateadd(minute, +15 , due_at) THEN delivery_id ELSE NULL END)
    AS scheduled_late_15min_count,

  count(distinct CASE WHEN delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS late_30min_count,
  count(distinct CASE WHEN delivery_type != 'pickup' AND delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS delivery_late_30min_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS pickup_late_30min_count,
  count(distinct CASE WHEN delivery_type = 'eta' AND delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS standard_eta_late_30min_count,
  count(distinct CASE WHEN delivery_type = 'priority_eta' AND delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS priority_eta_late_30min_count,
  count(distinct CASE WHEN delivery_type IN ('scheduled', 'limited_availability') AND delivered_at > dateadd(minute, +30 , due_at) THEN delivery_id ELSE NULL END)
    AS scheduled_late_30min_count,

  count(distinct CASE WHEN planned_delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS planned_late_count,
  count(distinct CASE WHEN delivery_type != 'pickup' AND planned_delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS delivery_planned_late_count,
  count(distinct CASE WHEN delivery_type = 'pickup' AND planned_delivered_at > due_at THEN delivery_id ELSE NULL END)
    AS pickup_planned_late_count,

  count(distinct CASE WHEN delivery_type IN ('scheduled', 'limited_availability')
                 AND delivered_at < window_starts_at
                 THEN delivery_id ELSE NULL END) as scheduled_early_count,

  round(sum(CASE WHEN delivery_type = 'eta' THEN DATEDIFF('seconds', created_at, delivered_at) ELSE 0 END) / 60) AS total_actual_wait_time_minutes_eta,
  round(sum(CASE WHEN delivery_type = 'priority_eta' THEN DATEDIFF('seconds', created_at, delivered_at) ELSE 0 END) / 60) AS total_actual_wait_time_minutes_priority_eta

  from deliveries_late d
  left join lateness_metrics lm
    on d.date = lm.date
    and d.zone_id = lm.zone_id
  group by 1, 2, 3, 4, 5, 6, 7, 8, 9
)






, batch_data as
(SELECT
       fb.zone_id,
       date_trunc('day', convert_timezone(z.time_zone_name, fb.delivery_completed_date_time_utc))::DATE date,
       fb.batch_id,
       count(*)                                                AS batch_whls,
       any_value(multi_warehouse_ind)                          AS multi_warehouse_ind,  
       sum(fb.num_deliveries)                                  AS batch_deliveries_full_service,
       round(sum(fb.active_time_seconds        ) / 60)         AS batch_active_minutes_full_service,
       round(sum(fb.in_store_time_seconds      ) / 60)         AS batch_in_store_minutes_full_service,
       round(max(fb.time_to_acknowledge_seconds) / 60)         AS batch_time_to_acknowledge_minutes_full_service,
       round(sum(fb.driving_time_seconds       ) / 60)         AS batch_time_to_warehouse_minutes_full_service,
       round(sum(fb.picking_time_seconds       ) / 60)         AS batch_picking_minutes_full_service,
       round(sum(fb.verification_time_seconds  ) / 60, 1)      AS batch_verification_minutes_full_service,
       round(sum(fb.cashiering_time_seconds    ) / 60)         AS batch_cashiering_minutes_full_service,
       round(sum(fb.delivery_time_seconds      ) / 60)         AS batch_delivery_minutes_full_service
  FROM dwh.fact_wloc_batch  fb
  JOIN rds_data.zones z
      ON z.id = fb.zone_id
 JOIN zone_dates zd on zd.zone_id = fb.zone_id and zd.date=date_trunc('day', convert_timezone(z.time_zone_name, fb.delivery_completed_date_time_utc))::DATE
  WHERE fb.num_deliveries > 0
    AND fb.batch_type IN ('delivery', 'rx_delivery' ,'drive_and_pick', 'delivery_only', 'pickup_delivery')
    AND fb.batch_state = 'completed'
    AND fb.shopper_assigned_to_batch_ind = 'Y'
  GROUP BY 1, 2, 3
)

, data_efficiency as
(SELECT
       zone_id,
       date,
 
       count(*)                                            AS batches_full_service, 
       sum(batch_deliveries_full_service)                  AS deliveries_full_service,
       sum(batch_whls)                                     AS batch_whls, 
       count_if(batch_deliveries_full_service = 1)         AS single_batches_full_service,
       count_if(batch_deliveries_full_service = 2)         AS double_batches_full_service,
       count_if(batch_deliveries_full_service = 3)         AS triple_batches_full_service,
       count_if(batch_deliveries_full_service = 4)         AS quad_batches_full_service,
       
       count_if(multi_warehouse_ind = 'N')                                       AS single_wh_batches_full_service,
       sum(case when multi_warehouse_ind = 'N' then batch_deliveries_full_service else 0 end) AS single_wh_deliveries_full_service,
       count_if(multi_warehouse_ind = 'N' and batch_deliveries_full_service = 2) AS single_wh_double_batches_full_service,
       count_if(multi_warehouse_ind = 'N' and batch_deliveries_full_service = 3) AS single_wh_triple_batches_full_service,
       count_if(multi_warehouse_ind = 'N' and batch_deliveries_full_service = 4) AS single_wh_quad_batches_full_service,
  
       count_if(multi_warehouse_ind = 'Y')                                       AS multi_wh_batches_full_service,
       sum(case when multi_warehouse_ind = 'Y' then batch_deliveries_full_service else 0 end) AS multi_wh_deliveries_full_service,
       count_if(multi_warehouse_ind = 'Y' and batch_deliveries_full_service = 2) AS multi_wh_double_batches_full_service,
       count_if(multi_warehouse_ind = 'Y' and batch_deliveries_full_service = 3) AS multi_wh_triple_batches_full_service,
       
       sum(batch_active_minutes_full_service)              AS active_minutes_full_service,
       sum(batch_in_store_minutes_full_service)            AS in_store_minutes_full_service,
       sum(batch_time_to_acknowledge_minutes_full_service) AS time_to_acknowledge_minutes_full_service,
       sum(batch_time_to_warehouse_minutes_full_service)   AS time_to_warehouse_minutes_full_service,
       sum(batch_picking_minutes_full_service)             AS picking_minutes_full_service,
       sum(batch_verification_minutes_full_service)        AS verification_minutes_full_service,
       sum(batch_cashiering_minutes_full_service)          AS cashiering_minutes_full_service,
       sum(batch_delivery_minutes_full_service)            AS delivery_minutes_full_service

  FROM batch_data
  GROUP BY 1, 2
)




, deliveries_cost AS
  (SELECT date_trunc('day', convert_timezone(z.time_zone_name, od.delivered_at))::DATE date,
          od.zone_id,
          count(*) AS deliveries,
          count(DISTINCT CASE WHEN od.delivery_type = 'priority_eta' THEN od.id ELSE NULL END) AS deliveries_priority_eta
   FROM rds_data.order_deliveries od
 INNER JOIN rds_data.zones z ON od.zone_id = z.id
   JOIN zone_dates zd on zd.zone_id = od.zone_id and zd.date=CONVERT_TIMEZONE('UTC', z.time_zone_name, od.delivered_at)::DATE
   WHERE od.workflow_state = 'delivered' 
   GROUP BY 1, 2)


,sum_costs AS
  (SELECT w.earned_date AS date_local,
          w.zone_id,
          sum(CASE WHEN w.wage_type = 'experiment_cost_adjustment' THEN amount_cents/100.0 ELSE 0 END) AS experiment_cost_adjustment_cost_dollars,
          sum(CASE WHEN w.wage_type IN ('driving_time', 'picking_time', 'delivery_time',
                                        'driving_mileage', 'mileage',
                                        'heavy_item_bump',
                                        'batch_payment_floor', 'batch_incentive_floor', 'add_on_batch_floor',
                                        'on_demand_acceptance_boost', 'ml_adjustment', 'time_adjustment_initial',
                                        'time_adjustment', 'random_adjustment', 'experiment_cost_adjustment',
                                        'peak_time_pay', 'hourly_base', 'service_bonus') THEN amount_cents/100.0 ELSE 0 END) AS total_labor_cost_dollars,
          sum(CASE WHEN w.wage_type IN ('driving_time', 'picking_time', 'delivery_time',
                                        'driving_mileage', 'mileage',
                                        'heavy_item_bump',
                                        'batch_payment_floor', 'batch_incentive_floor', 'add_on_batch_floor',
                                        'on_demand_acceptance_boost', 'ml_adjustment', 'time_adjustment_initial',
                                        'time_adjustment', 'random_adjustment', 'experiment_cost_adjustment',
                                        'peak_time_pay') THEN amount_cents/100.0 ELSE 0 END) AS fs_labor_cost_dollars,
          sum(CASE WHEN w.wage_type = 'driving_time' THEN amount_cents/100.0 ELSE 0 END) AS driving_time_cost_dollars,
          sum(CASE WHEN w.wage_type = 'picking_time' THEN amount_cents/100.0 ELSE 0 END) AS picking_time_cost_dollars,
          sum(CASE WHEN w.wage_type = 'delivery_time' THEN amount_cents/100.0 ELSE 0 END) AS delivery_time_cost_dollars,
          sum(CASE WHEN w.wage_type = 'driving_mileage' THEN amount_cents/100.0 ELSE 0 END) AS driving_mileage_cost_dollars,
          sum(CASE WHEN w.wage_type = 'mileage' THEN amount_cents/100.0 ELSE 0 END) AS mileage_cost_dollars,
          sum(CASE WHEN w.wage_type = 'heavy_item_bump' THEN amount_cents/100.0 ELSE 0 END) AS heavy_item_bump_cost_dollars,
          sum(CASE WHEN w.wage_type = 'batch_payment_floor' THEN amount_cents/100.0 ELSE 0 END) AS batch_payment_floor_cost_dollars,
          sum(CASE WHEN w.wage_type = 'batch_incentive_floor' THEN amount_cents/100.0 ELSE 0 END) AS batch_incentive_floor_cost_dollars,
          sum(CASE WHEN w.wage_type = 'add_on_batch_floor' THEN amount_cents/100.0 ELSE 0 END) AS add_on_batch_floor_cost_dollars,
          sum(CASE WHEN w.wage_type = 'on_demand_acceptance_boost' THEN amount_cents/100.0 ELSE 0 END) AS on_demand_acceptance_boost_cost_dollars,
          sum(CASE WHEN w.wage_type = 'ml_adjustment' THEN amount_cents/100.0 ELSE 0 END) AS ml_adjustment_cost_dollars,
          sum(CASE WHEN w.wage_type IN ('time_adjustment_initial', 'time_adjustment') THEN amount_cents/100.0 ELSE 0 END) AS time_adjustment_cost_dollars,
          sum(CASE WHEN w.wage_type = 'random_adjustment' THEN amount_cents/100.0 ELSE 0 END) AS random_adjustment_cost_dollars,
          sum(CASE WHEN w.wage_type = 'peak_time_pay' THEN amount_cents/100.0 ELSE 0 END) AS peak_time_pay_cost_dollars,
          sum(CASE WHEN w.wage_type = 'hourly_base' THEN amount_cents/100.0 ELSE 0 END) AS hourly_base_cost_dollars,
          sum(CASE WHEN w.wage_type = 'service_bonus' THEN amount_cents/100.0 ELSE 0 END) AS service_bonus_cost_dollars,
          sum(CASE WHEN w.wage_type = 'hourly_bonus' THEN amount_cents/100.0 ELSE 0 END) AS hourly_bonus_cost_dollars,          
          sum(CASE WHEN w.wage_type = 'quest_bonus' THEN amount_cents/100.0 ELSE 0 END) AS quest_bonus_cost_dollars,    
          sum(CASE WHEN w.wage_type = 'tip' THEN amount_cents/100.0 ELSE 0 END) AS tip_dollars
   FROM rds_data.wages w
   INNER JOIN rds_data.zones z ON w.zone_id = z.id
    JOIN zone_dates zd on zd.zone_id = w.zone_id and zd.date=w.earned_date
    WHERE w.deleted_ind = 'N'
   GROUP BY 1, 2
)


, sum_costs_priority_eta AS (
     SELECT w.earned_date AS date_local,
            w.zone_id,
            sum(CASE WHEN w.wage_type = 'experiment_cost_adjustment' AND od.delivery_type='priority_eta' THEN amount_cents/100.0 ELSE 0 END) AS experiment_cost_adjustment_cost_dollars_priority_eta,
            sum(CASE WHEN w.wage_type IN ('driving_time', 'picking_time', 'delivery_time',
                                        'driving_mileage', 'mileage',
                                        'heavy_item_bump',
                                        'batch_payment_floor', 'batch_incentive_floor', 'add_on_batch_floor',
                                        'on_demand_acceptance_boost', 'ml_adjustment', 'time_adjustment_initial',
                                        'time_adjustment', 'random_adjustment', 'experiment_cost_adjustment',
                                        'peak_time_pay', 'hourly_base', 'service_bonus')
                          AND od.delivery_type='priority_eta' THEN amount_cents/100.0 ELSE 0 END) AS total_labor_cost_dollars_priority_eta,
            sum(CASE WHEN w.wage_type IN ('driving_time', 'picking_time', 'delivery_time',
                                        'driving_mileage', 'mileage',
                                        'heavy_item_bump',
                                        'batch_payment_floor', 'batch_incentive_floor', 'add_on_batch_floor',
                                        'on_demand_acceptance_boost', 'ml_adjustment', 'time_adjustment_initial',
                                        'time_adjustment', 'random_adjustment', 'experiment_cost_adjustment',
                                        'peak_time_pay')
                          AND od.delivery_type='priority_eta' THEN amount_cents/100.0 ELSE 0 END) AS fs_labor_cost_dollars_priority_eta,
            sum(CASE WHEN w.wage_type = 'on_demand_acceptance_boost' AND od.delivery_type='priority_eta' THEN amount_cents/100.0 ELSE 0 END) AS on_demand_acceptance_boost_cost_dollars_priority_eta
   FROM rds_data.wages w
   INNER JOIN rds_data.zones z ON w.zone_id = z.id
   LEFT JOIN rds_data.order_delivery_batches odb
   ON odb.batch_id = w.batch_id
   LEFT JOIN rds_data.order_deliveries od
   ON odb.order_delivery_id = od.id
     JOIN zone_dates zd on w.zone_id = zd.zone_id and w.earned_date=zd.date
   WHERE w.deleted_ind = 'N'
   
   GROUP BY 1, 2)




, data_cost as
(
SELECT d.zone_id,
       d.date as date,
       d.deliveries,
       d.deliveries_priority_eta,
       c.total_labor_cost_dollars,
       c.fs_labor_cost_dollars,
       c.driving_mileage_cost_dollars,
       c.mileage_cost_dollars,
       c.picking_time_cost_dollars,
       c.batch_payment_floor_cost_dollars,
       c.batch_incentive_floor_cost_dollars,
       c.heavy_item_bump_cost_dollars,
       c.on_demand_acceptance_boost_cost_dollars,
       c.add_on_batch_floor_cost_dollars,
       c.hourly_base_cost_dollars,
       c.service_bonus_cost_dollars,
       c.ml_adjustment_cost_dollars,
       c.time_adjustment_cost_dollars,
       c.random_adjustment_cost_dollars,
       c.driving_time_cost_dollars,
       c.delivery_time_cost_dollars,
       c.hourly_bonus_cost_dollars, 
       c.quest_bonus_cost_dollars, 
       c.tip_dollars,
       c.experiment_cost_adjustment_cost_dollars,
       c.peak_time_pay_cost_dollars,
       cpeta.experiment_cost_adjustment_cost_dollars_priority_eta,
       cpeta.total_labor_cost_dollars_priority_eta,
       cpeta.fs_labor_cost_dollars_priority_eta,
       cpeta.on_demand_acceptance_boost_cost_dollars_priority_eta
FROM deliveries_cost d
INNER JOIN sum_costs c ON (c.date_local = d.date AND d.zone_id = c.zone_id)
LEFT JOIN sum_costs_priority_eta cpeta ON (cpeta.date_local = d.date AND cpeta.zone_id = d.zone_id)
)



, reschedules as (
    select
        order_delivery_id,
        max(rescheduled_at) as rescheduled_at
    from rds_data.order_delivery_reschedules
    where created_at in (select date from zone_dates)
    group by 1)

, deliveries_tta AS  (
    SELECT  od.zone_id,
            DATE_TRUNC('day', CONVERT_TIMEZONE(z.time_zone_name, od.delivered_at))::DATE AS date,
            od.id,
         od.delivery_type,
            datediff('second', MIN(ps.assigned_at), MAX(ps.delivery_acknowledged_at)) AS time_to_accept_seconds
     FROM rds_data.order_delivery_batches odb
     INNER JOIN rds_data.order_deliveries od ON od.id = odb.order_delivery_id
     INNER JOIN rds_data.zones z ON od.zone_id = z.id
     INNER JOIN rds_data.picking_sets ps ON ps.id = odb.batch_id
        AND ps.batch_type IN ('delivery_only','delivery')
       -- AND DATE_TRUNC('day', CONVERT_TIMEZONE(z.time_zone_name, od.delivered_at))::DATE between cast('2021-06-24' as date) and cast('2021-07-14' as date)
    JOIN zone_dates zd on zd.zone_id = od.zone_id and zd.date=CONVERT_TIMEZONE('UTC', z.time_zone_name, od.delivered_at)::DATE
     LEFT JOIN reschedules as r ON od.id = r.order_delivery_id
     WHERE TRUE
        AND od.workflow_state = 'delivered'
        AND r.rescheduled_at IS NULL
     GROUP BY 1, 2, 3, 4
     HAVING datediff('minute', MIN(odb.created_at), MAX(odb.created_at)) >= 0
)

, zone_tta_stats_new AS (
    SELECT date,
           zone_id,
           --variant,
           count(*)                                                                    as deliveries,
           avg(time_to_accept_seconds) / 60                                            as avg_time_to_accept_per_delivery,
           approx_percentile(time_to_accept_seconds, 0.85)                             as p85_time_to_accept_per_delivery,
           STDDEV(time_to_accept_seconds/60) as stddev_tta,
            count(DISTINCT CASE WHEN delivery_type='priority_eta' THEN id ELSE NULL END) as deliveries_priority_eta,
    --   avg(time_to_accept_seconds)/60.0 as avg_time_to_accept_per_delivery, 
       zeroifnull(avg(CASE WHEN delivery_type='priority_eta' THEN time_to_accept_seconds ELSE NULL END))/60.0 as avg_time_to_accept_per_delivery_priority_eta,
     --  approx_percentile(time_to_accept_seconds, 0.85) as p85_time_to_accept_per_delivery
           (percentile_cont(0.25) WITHIN GROUP (ORDER BY time_to_accept_seconds)) / 60 as tta_percentile_25,
           (percentile_cont(0.5) WITHIN GROUP (ORDER BY time_to_accept_seconds)) / 60  as tta_median,
           (percentile_cont(0.75) WITHIN GROUP (ORDER BY time_to_accept_seconds)) / 60 as tta_percentile_75,
           (percentile_cont(0.90) WITHIN GROUP (ORDER BY time_to_accept_seconds)) / 60 as tta_percentile_90
    FROM deliveries_tta
    GROUP BY 1, 2--,3
)


,current_hourly_rates as (select wr.*, zd.variant, zd.date from rds_data.wage_rates wr
join (select zone_id, max(starts_on) as latest_starts_on from rds_data.wage_rates where rate_type = 'active_time' group by 1)
dedupe_key on dedupe_key.zone_id=wr.zone_id and dedupe_key.latest_starts_on = wr.starts_on
join zone_dates zd ON wr.zone_id=zd.zone_id
where rate_type = 'active_time'
order by amount_cents desc)

, logistics_dpah as
(SELECT
       fb.zone_id,
       date_trunc('day', convert_timezone(z.time_zone_name, fb.delivery_completed_date_time_utc))::DATE date,

       count(*)               AS batches_full_service,
       sum(fb.num_deliveries) AS deliveries_full_service,

       count_if(fb.num_deliveries = 1) AS single_batches_full_service,
       count_if(fb.num_deliveries = 2) AS double_batches_full_service,
       count_if(fb.num_deliveries = 3) AS triple_batches_full_service,
       count_if(fb.num_deliveries = 4) AS quad_batches_full_service,

       round(sum(fb.active_time_seconds        ) / 60) AS active_minutes_full_service,
       round(sum(fb.in_store_time_seconds      ) / 60) AS in_store_minutes_full_service,

       round(sum(fb.time_to_acknowledge_seconds) / 60) AS time_to_acknowledge_minutes_full_service,
       round(sum(fb.time_to_warehouse_seconds  ) / 60) AS time_to_warehouse_minutes_full_service,
       round(sum(fb.picking_time_seconds       ) / 60) AS picking_minutes_full_service,
       round(sum(fb.verification_time_seconds  ) / 60, 1) AS verification_minutes_full_service,
       round(sum(fb.cashiering_time_seconds    ) / 60) AS cashiering_minutes_full_service,
       round(sum(fb.delivery_time_seconds      ) / 60) AS delivery_minutes_full_service

  FROM dwh.fact_batch  fb
  JOIN rds_data.zones z
      ON z.id = fb.zone_id
  JOIN zone_dates zd ON fb.zone_id=zd.zone_id and date_trunc('day', convert_timezone(z.time_zone_name, fb.delivery_completed_date_time_utc))::DATE = zd.date

  WHERE fb.num_deliveries > 0
    AND fb.batch_type = 'delivery'
    AND fb.batch_state = 'completed'
    AND fb.delivery_canceled_at_ind = 'N'
    AND fb.shopper_shift_type = 'normal'
 
  GROUP BY 1, 2
)






    SELECT cb.date
         , zd.zone_id
         , zd.variant
         , r.id                                                                    as region_id
         
  --late metrics
   , 100 * planned_late_count / (deliveries_count + 0.001) as  planned_late_percent
    , 100 * late_count / (deliveries_count + 0.001) as late_percent 
      , avg_lateness_minutes
       , median_lateness_minutes
        , p85_lateness_minutes
        , p90_lateness_minutes
        , p95_lateness_minutes
       , p99_lateness_minutes
        ,100 * scheduled_early_count / (scheduled_deliveries_count + 0.001) as scheduled_early_percent
     , 100 * pickup_late_count / (pickup_deliveries_count + 0.001) as pickup_late_percent
    , 100 * delivery_late_count / (delivery_deliveries_count + 0.001) as delivery_late_percent
        , 100 * pickup_planned_late_count / (pickup_deliveries_count + 0.001) as  pickup_planned_late_percent
        
      ,  100 * delivery_planned_late_count / (delivery_deliveries_count + 0.001) as delivery_planned_late_percent
        ,  100 * standard_eta_late_count / (standard_eta_deliveries_count + 0.001) as standard_eta_late_percent
        ,  100 * priority_eta_late_count / (priority_eta_deliveries_count + 0.001) as priority_eta_late_percent  
      ,  100 * scheduled_late_count / (scheduled_deliveries_count + 0.001) as scheduled_late_percent
        , 100 * delivery_late_15min_count / (delivery_deliveries_count + 0.001) as delivery_15_minute_late_percent 
            
     , 100 * standard_eta_late_15min_count / (standard_eta_deliveries_count + 0.001) as standard_eta_15_minute_late_percent
        , 100 * priority_eta_late_15min_count / (priority_eta_deliveries_count + 0.001) as  priority_eta_15_minute_late_percent
        , 100 * scheduled_late_15min_count / (scheduled_deliveries_count + 0.001) as scheduled_15_minute_late_percent
       , 100 * delivery_late_30min_count / (delivery_deliveries_count + 0.001) as delivery_30_minute_late_percent
       ,  100 * standard_eta_late_30min_count / (standard_eta_deliveries_count + 0.001) as standard_eta_30_minute_late_percent
                 
      , 100 * priority_eta_late_30min_count / (priority_eta_deliveries_count + 0.001) as priority_eta_30_minute_late_percent
       
     , 100 * scheduled_late_30min_count / (scheduled_deliveries_count + 0.001) as scheduled_30_minute_late_percent
     , total_actual_wait_time_minutes_eta / (standard_eta_deliveries_count + 0.001) as average_fulfillment_minutes_standard_eta     
    , total_actual_wait_time_minutes_priority_eta / (priority_eta_deliveries_count + 0.001) as average_fulfillment_minutes_priority_eta
    
    
    
    --cost metrics
    , 60.0 * fs_labor_cost_dollars / (active_minutes_full_service + 0.001) as fs_average_hourly_rate
  ,  total_labor_cost_dollars / cb.deliveries as total_cpd
      , fs_labor_cost_dollars / cb.deliveries as fs_cpd
      , total_labor_cost_dollars_priority_eta / (cb.deliveries_priority_eta + 0.001) as total_priority_eta_cpd
      , fs_labor_cost_dollars_priority_eta / (cb.deliveries_priority_eta + 0.001) as fs_priority_eta_cpd 
      , driving_time_cost_dollars / cb.deliveries as driving_time_cpd
      , driving_mileage_cost_dollars / cb.deliveries as driving_mileage_cpd
      , picking_time_cost_dollars / cb.deliveries as picking_time_cpd
      , delivery_time_cost_dollars / cb.deliveries as delivery_time_cpd
      , mileage_cost_dollars / cb.deliveries     as mileage_cpd   
      , batch_payment_floor_cost_dollars / cb.deliveries as batch_payment_floor_cpd
      , batch_incentive_floor_cost_dollars / cb.deliveries as batch_incentive_floor_cpd
      , heavy_item_bump_cost_dollars / cb.deliveries as heavy_item_bump_cpd
      , on_demand_acceptance_boost_cost_dollars / cb.deliveries as on_demand_acceptance_boost_cpd
      , on_demand_acceptance_boost_cost_dollars_priority_eta / (cb.deliveries_priority_eta + 0.001) as on_demand_acceptance_boost_priority_eta_cpd
      , add_on_batch_floor_cost_dollars / cb.deliveries as add_on_batch_floor_cpd
      , hourly_base_cost_dollars / cb.deliveries as hourly_base_cpd
      , service_bonus_cost_dollars / cb.deliveries as service_bonus_cpd
      , ml_adjustment_cost_dollars / cb.deliveries as ml_adjustment_cpd
      , time_adjustment_cost_dollars / cb.deliveries as time_adjustment_cpd
      , random_adjustment_cost_dollars / cb.deliveries as random_adjustment_cpd
      , hourly_bonus_cost_dollars/cb.deliveries as hourly_bonus_cpd
       , quest_bonus_cost_dollars/cb.deliveries as quest_bonus_cpd 
       , tip_dollars/cb.deliveries as tip_per_del
       , experiment_cost_adjustment_cost_dollars/ cb.deliveries as experiment_cost_adjustment_cpd
       , peak_time_pay_cost_dollars / cb.deliveries as peak_time_pay_cpd
       , experiment_cost_adjustment_cost_dollars_priority_eta/(cb.deliveries_priority_eta + 0.001)  as experiment_cost_adjustment_priority_eta_cpd
       
       
       --tta metrics
     , avg_time_to_accept_per_delivery 
     , avg_time_to_accept_per_delivery_priority_eta
     , tta_percentile_25 
     , tta_percentile_75
     , tta_percentile_90 
     , tta_median 
     
     --weights
     
     , deliveries_count
     , delivery_deliveries_count
     ,pickup_deliveries_count
     ,fss_pickup_deliveries_count
     , partner_pickup_deliveries_count    
 , standard_eta_deliveries_count
 ,priority_eta_deliveries_count
 ,scheduled_deliveries_count
  ,active_minutes_full_service 
  ,cb.deliveries 
  ,cb.deliveries_priority_eta 
     
       
       
       
       
       
     
    FROM data_cost cb
             LEFT JOIN zone_dates zd ON cb.zone_id = zd.zone_id AND cb.date = zd.date
             left join data_late dl ON cb.zone_id = dl.zone_id AND cb.date = dl.date
             left join data_efficiency de ON cb.zone_id = de.zone_id AND cb.date = de.date
             left join zone_tta_stats_new zts ON cb.zone_id = zts.zone_id AND cb.date = zts.date
           --  left join pred_times pt ON cb.zone_id = pt.zone_id AND cb.date = pt.date
            -- left join junk_batches jb ON cb.zone_id = jb.zone_id AND cb.date = jb.date
             LEFT JOIN RDS_DATA.ZONES Z ON Z.ID = ZD.ZONE_ID
             LEFT JOIN RDS_DATA.REGIONS R ON R.ID = Z.REGION_ID
            -- LEFT JOIN logistics_dpah ldpah ON ldpah.zone_id=zd.zone_id and ldpah.date=zd.date


--LEFT JOIN current_hourly_rates ch  ON cb.zone_id=ch.zone_id AND cb.date=ch.date
    WHERE 1 = 1

   -- GROUP BY 1, 2, 3
    ORDER BY 1
"""

In [86]:
noncost_query_2="""with zone_dates as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-03-20' as date)
    ORDER BY 1
    )
    
    -- The table starts from 03/16/2022 so I cannot use bad batch impressions for this experiments which starts in February.
, bad_batch_impressions as
(
  select
      session_start_local::date as date
      , bb.zone_id
      , sum(bad_batch) as bad_batch_impressions
      , sum(very_bad_batch) as very_bad_batch_impressions
      , count(bad_batch) as batch_impressions
      , count(distinct case when bad_batch is not null then batch_id else null end) as batches
      , bad_batch_impressions/batch_impressions as bad_batch_imp_pct
      , very_bad_batch_impressions/batch_impressions as very_bad_batch_imp_pct
  from analysts.bad_batch_impressions bb
JOIN zone_dates zd ON bb.zone_id = zd.zone_id AND bb.session_start_local::date = zd.date
  group by 1, 2)
  
  
, lost_deliveries_initial AS (
SELECT
    dem.date,
    customer_zone_id AS zone_id,
    sum(total_lost_demand) total_lost_demand,
    sum(deliveries_created) total_deliveries_created,
    sum(total_lost_demand + deliveries_created) total_demand,  
    sum(total_capacity_lost_demand) as total_capacity_lost_demand
    
FROM  dwh.demand_estimates_metrics dem
JOIN zone_dates zd ON dem.customer_zone_id = zd.zone_id AND dem.date = zd.date
WHERE 1=1
AND deleted_ind = 'N'
GROUP BY 1, 2
)

, lost_deliveries as(SELECT date, zone_id, total_demand,
                     total_lost_demand/total_demand as lost_demand_pct,
    total_capacity_lost_demand/total_demand as capacity_lost_demand_pct    
                     FROM  lost_deliveries_initial)
  
  
SELECT zd.date, zd.zone_id, lost_demand_pct,capacity_lost_demand_pct, total_demand   
--,bad_batch_imp_pct, very_bad_batch_imp_pct
FROM zone_dates zd 
left join lost_deliveries dl ON zd.zone_id = dl.zone_id AND zd.date = dl.date"""

In [87]:
noncost_query_3="""
WITH zone_dates_590 as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 590)
    
     , zone_dates_581 as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 581
                         )
                         
     , zone_dates_607 as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 607    
                         )
    
    
    , zone_dates as (select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-03-20' as date)
    ORDER BY 1
    )
    
   SELECT zd1.date, zd1.zone_id, CASE WHEN zd2.variant IS NULL THEN '0' 
    WHEN zd2.variant='control' THEN '0' 
    WHEN zd2.variant='variant' THEN '1' END as exp_590
    ,CASE WHEN zd3.variant IS NULL THEN '0' 
    WHEN zd3.variant='control' THEN '0' 
    WHEN zd3.variant='item_rel' THEN '1' 
    WHEN zd3.variant='item_rel_multi' THEN '2' 
    WHEN zd3.variant='multi' THEN '3' 
    END as exp_581   
     ,CASE WHEN zd4.variant IS NULL THEN '0' 
    WHEN zd4.variant='control' THEN '0' 
    WHEN zd4.variant='variant' THEN '1' END as exp_607
    
    FROM zone_dates zd1
   LEFT JOIN zone_dates_590 zd2 ON zd1.zone_id=zd2.zone_id AND zd1.date=zd2.date
   LEFT JOIN zone_dates_581 zd3 ON zd1.zone_id=zd3.zone_id AND zd1.date=zd3.date
   LEFT JOIN zone_dates_607 zd4 ON zd1.zone_id=zd4.zone_id AND zd1.date=zd4.date
"""

In [88]:
noncost_query_4="""
--2016 rows: 96 zones * 21 days
--672 rows for each of the 3 variants
WITH experiment_schedule as (
    select date
         , zone_id
         , z.name as zone_name
         , variant
    from rds_data.logistics_experiments_schedules les
             join rds_data.zones z on z.id = les.zone_id
    where experiment_id = 580
      AND date BETWEEN cast('2022-02-07' as date) and cast('2022-03-20' as date)
    ORDER BY 1)

-- Simulation output ""...cents" columns are daily totals of all listed batches in the batch_ids column of the given shopper-day.
   , simulation_output AS (
    select driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents
           --         
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation WHERE experiment_id='580')

   , sim_output_batches AS (
    select DISTINCT driver_id,
           date::date as date,
           guaranteed_minimum_amount_cents,
           batch_payment_amount_cents,
           daily_adjustment_amount_cents,
           f.value    as batch_id
    from segment_events.insta_prod_shoppers.ca_zone_day_experimentation
       , table (flatten(input => parse_json(batch_ids))) f WHERE experiment_id='580')
       
       ,weekly_adjustment_data as (
      select
          wa.*
          ,parse_json(data):batches x
      from rds_data.weekly_adjustments wa
      LEFT JOIN INSTADATA.RDS_DATA.PAY_PERIODS pp ON wa.PAY_PERIOD_ID = pp.ID
      INNER JOIN
        ( SELECT driver_id, PAY_PERIOD_ID, MAX(UPDATED_AT) UPDATED_AT
          FROM rds_data.weekly_adjustments
          GROUP BY 1,2
        ) dedupe_key
        ON wa.DRIVER_ID = dedupe_key.DRIVER_ID
        AND wa.PAY_PERIOD_ID = dedupe_key.PAY_PERIOD_ID
        AND wa.UPDATED_AT = dedupe_key.UPDATED_AT

      WHERE TRUE
        AND pp.STARTS_AT::DATE >= '2022-02-07'
)

, wa_batch_counted AS (
    SELECT
        value:batch_id as batch_id,
        value:distance_miles_ceiling as batch_miles_EG,
        value:duration_minutes_ceiling as batch_mins_EG,
        value:min_wage_rate_amount_cents / 100 as min_wage,
        1.2 * min_wage * (batch_mins_EG / 60) as batch_time_EG_pay,
        batch_miles_EG * 0.3 as batch_distance_EG_pay,
        batch_time_EG_pay+batch_distance_EG_pay as eg_pay,
        ANY_VALUE(driver_id) as driver_id,
        ANY_VALUE(pay_period_id) as pay_period_id,
        ANY_VALUE(guaranteed_minimum_amount_cents / 100) as guarantee_pay
    FROM weekly_adjustment_data
        ,lateral flatten(input => x)
    GROUP BY 1,2,3,4
    ORDER BY 4 desc
)       

   , zone_day AS (
    SELECT s.date   as date,
           --s.driver_id,
         --  rs.shopper_level,
           es.variant,
           ps.zone_id,
           z.name   as zone_name,
           z.state  as state,
           r.id     as region_id,
           r.name   as region_name,
           c.name   as country,
        SUM(eg_pay) as total_eg_pay, 
   
           SUM(ZEROIFNULL(fb.n_del)) as del_number,
        total_eg_pay/del_number as guaranteed_pay_per_del,
           sum(case
                   when ps.batch_type = 'drive_and_pick' then
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_started_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
                   else
                       greatest(LEAST(datediff('seconds', ps.delivery_acknowledged_at,
                                               coalesce(ps.delivery_completed_at, ps.picking_completed_at,
                                                        ps.picking_started_at, ps.delivery_acknowledged_at)) / 3600,
                                      12), 0)
               end) AS active_hours
           --  , SUM(s.batch_payment_amount_cents)/100 as zone_day_upfront_pay
           --  ,SUM(s.guaranteed_minimum_amount_cents)/100 as zone_day_guarantee_pay
    FROM rds_data.picking_sets ps
             JOIN rds_data.zones z ON (z.id = ps.zone_id)
             JOIN rds_data.regions r ON r.id = z.region_id
             JOIN rds_data.countries c ON c.id = r.country_id
             JOIN (select batch_id, sum(num_deliveries) as n_del from dwh.fact_wloc_batch fwb WHERE SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y' group by 1) fb ON fb.batch_id=ps.id
             JOIN sim_output_batches s ON s.batch_id = ps.id
             JOIN  wa_batch_counted w ON w.batch_id = s.batch_id
             JOIN experiment_schedule es ON es.zone_id = ps.zone_id AND es.date = s.date
           --  LEFT JOIN (select distinct batch_date,
           --                             shopper_id,
           --                             iff(running_total_batches < 11, 'very new', level) as shopper_level
           --             from analysts.shopper_retention_engagement) rs on rs.shopper_id = ps.driver_id and
           --                                                               rs.batch_date =
           --                                                               convert_timezone(z.time_zone_name,
                                                                                           --ps.delivery_completed_at)::date
    WHERE 1 = 1
          --     AND ps.workflow_state = 'completed'
          --  AND fb.SHOPPER_ASSIGNED_TO_BATCH_IND = 'Y'
          -- AND ps.batch_type IN ('delivery', 'rx_delivery', 'drive_and_pick', 'delivery_only', 'pickup_delivery')
          --AND CONVERT_TIMEZONE(z.time_zone_name, ps.delivery_completed_at)::DATE between cast('2021-07-26' as date) and cast('2021-11-14' as date)
          -- AND ps.zone_id in (select zone_id from asp_eligible_zone_list)
    GROUP BY 1, 2, 3, 4, 5, 6, 7,8)
    
    
    SELECT zone_id, date, variant, guaranteed_pay_per_del
    FROM zone_day
"""

## 2. Analysis of the Cost Metrics 

This section covers the analysis of the overpay and guaranteed pay metrics per delivery that need a special readout analysis compared to other metrics.

### 2.1. Data Preparation

In [89]:
# Create a dataframe for each of the three-week clusters of overpay data defined above
df_overpay1=iq.query(overpay_query_1)
df_overpay2=iq.query(overpay_query_2)

In [90]:
# Add an indicator for the given three-week cluster into each dataframe  
df_overpay1['cluster']=1
df_overpay2['cluster']=2

In [91]:
# Join the overpay clusters in one dataframe
df1= pd.concat([df_overpay1, df_overpay2])
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 287
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   zone_id           576 non-null    int16 
 1   region_id         576 non-null    int16 
 2   variant           576 non-null    object
 3   overpay_per_del   576 non-null    object
 4   total_deliveries  576 non-null    object
 5   cluster           576 non-null    int64 
dtypes: int16(2), int64(1), object(3)
memory usage: 24.8+ KB


In [92]:
# Create a dataframe for each of the three-week clusters of guaranteed pay data defined above
df_guaranteedpay1=iq.query(guaranteed_pay_query_1)
df_guaranteedpay2=iq.query(guaranteed_pay_query_2)

In [93]:
# Add an indicator for the given three-week cluster into each dataframe  
df_guaranteedpay1['cluster']=1
df_guaranteedpay2['cluster']=2

In [94]:
# Join the guaranteed pay clusters in one dataframe
df2= pd.concat([df_guaranteedpay1, df_guaranteedpay2])
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 287
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   zone_id                 576 non-null    int16  
 1   variant                 576 non-null    object 
 2   guaranteed_pay_per_del  576 non-null    float64
 3   cluster                 576 non-null    int64  
dtypes: float64(1), int16(1), int64(1), object(1)
memory usage: 19.1+ KB


In [95]:
# Join the overpay and guaranteed pay data on zone_id, variant and cluster
df_base= pd.merge(df1, df2, on=["zone_id", "variant","cluster"])
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   zone_id                 576 non-null    int16  
 1   region_id               576 non-null    int16  
 2   variant                 576 non-null    object 
 3   overpay_per_del         576 non-null    object 
 4   total_deliveries        576 non-null    object 
 5   cluster                 576 non-null    int64  
 6   guaranteed_pay_per_del  576 non-null    float64
dtypes: float64(1), int16(2), int64(1), object(3)
memory usage: 29.2+ KB


In [96]:
# Convert the overpay_per_del and total_deliveries to numeric data
df_base['overpay_per_del']=pd.to_numeric(df_base['overpay_per_del'])
df_base['total_deliveries']=pd.to_numeric(df_base['total_deliveries'])
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 576 entries, 0 to 575
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   zone_id                 576 non-null    int16  
 1   region_id               576 non-null    int16  
 2   variant                 576 non-null    object 
 3   overpay_per_del         576 non-null    float64
 4   total_deliveries        576 non-null    float64
 5   cluster                 576 non-null    int64  
 6   guaranteed_pay_per_del  576 non-null    float64
dtypes: float64(3), int16(2), int64(1), object(1)
memory usage: 29.2+ KB


In [97]:
# Create 60 region-clusters (30 regions * 2 clusters)
df_base['region_cluster']=df_base.groupby(['region_id', 'cluster']).ngroup()
df_base['region_cluster']

0       2
1       2
2      10
3       2
4      50
       ..
571     5
572     3
573    31
574    17
575     1
Name: region_cluster, Length: 576, dtype: int64

In [98]:
# Create two dataframes where each dataframe contains the control and one of the treatments 
df_v1=df_base[(df_base['variant']=='control')|(df_base['variant']=='variant1')]
df_v2=df_base[(df_base['variant']=='control')|(df_base['variant']=='variant2')]
df_v1

,zone_id,region_id,variant,overpay_per_del,total_deliveries,cluster,guaranteed_pay_per_del,region_cluster
0,1053,7,control,0.000000,567.0,1,20.496599,2
1,1075,7,control,0.004594,975.0,1,15.954225,2
2,1051,143,control,0.083402,1649.0,1,17.123007,10
3,618,7,control,0.168852,9141.0,1,14.963698,2
4,1055,366,control,0.002036,781.0,1,20.545665,50
...,...,...,...,...,...,...,...,...
568,1910,51,control,0.000431,209.0,2,25.290404,9
569,1076,155,variant1,0.097165,187.0,2,20.002085,15
573,832,225,control,0.015691,2999.0,2,14.059954,31
574,561,157,variant1,0.049620,5800.0,2,14.133061,17


## 2.2. Comparison of the Control and Variant 1

Variant 1 increases the discount factor in a given zone by 5 pp.

In [99]:
# Divide the data into control and treatment
df=df_v1
control_data=df[df['variant']=='control']
treatment_data=df[df['variant']=='variant1']

In [100]:
# Calculate the coefficient and confidence interval for the cost metrics
metrics= {
 'overpay_per_del':'total_deliveries',
'guaranteed_pay_per_del':'total_deliveries'}

results=pd.DataFrame(index=metrics.keys(), columns=['control_mean','estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric, weight in metrics.items():
    equation = '%s ~ variant+C(cluster)'
    model = smf.wls( equation % metric, data=df, weights=df[weight]).fit()
    #Using robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=df['region_cluster'])
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['control_mean']=DescrStatsW(control_data[metric], weights=control_data[weight]).mean
    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

,control_mean,estimate,p-value,[0.025,0.975],significance
overpay_per_del,0.064498,0.038295,0.0,0.025092,0.051498,significant
guaranteed_pay_per_del,15.440916,0.015004,0.701035,-0.062817,0.092825,not significant


## 2.3. Comparison of the Control and Variant 2

Variant 2 decreases the discount factor in a given zone by 5 pp.

In [101]:
# Divide the data into control and treatment for variant 2
df=df_v2
control_data=df[df['variant']=='control']
treatment_data=df[df['variant']=='variant2']

In [102]:
# Calculate the coefficient and confidence interval for several metrics
metrics= {
 'overpay_per_del':'total_deliveries',
'guaranteed_pay_per_del':'total_deliveries'}

results=pd.DataFrame(index=metrics.keys(), columns=['control_mean','estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric, weight in metrics.items():
    equation = '%s ~ variant+C(cluster)'
    model = smf.wls( equation % metric, data=df, weights=df[weight]).fit()
    #Using robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=df['region_cluster'])
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['control_mean']=DescrStatsW(control_data[metric], weights=control_data[weight]).mean
    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

,control_mean,estimate,p-value,[0.025,0.975],significance
overpay_per_del,0.064498,-0.027849,0.000001,-0.03799,-0.017707,significant
guaranteed_pay_per_del,15.440916,0.055048,0.008726,0.014446,0.09565,significant


# 3. Analysis of Non-Cost Metrics

This section covers the analysis of all the metrics that are not overpay or guarateed pay.

## 3.1. Data Preparation

In [103]:
# Create a dataframe from the noncost_query defined above
df3=iq.query(noncost_query)
df3.head()

,date,zone_id,variant,region_id,planned_late_percent,late_percent,avg_lateness_minutes,median_lateness_minutes,p85_lateness_minutes,p90_lateness_minutes,...,delivery_deliveries_count,pickup_deliveries_count,fss_pickup_deliveries_count,partner_pickup_deliveries_count,standard_eta_deliveries_count,priority_eta_deliveries_count,scheduled_deliveries_count,active_minutes_full_service,deliveries,deliveries_priority_eta
0,2022-02-07,1920,variant1,191,3.658492,13.414471,-48.506329,-55.000,-4.400,5.800,...,82,0,0,0,50,27,5,3974,82,27
1,2022-02-07,2234,control,1,33.327779,49.991668,-39.166667,-24.500,35.500,49.000,...,6,0,0,0,0,0,2,438,6,0
2,2022-02-07,1925,variant1,143,24.998958,20.832465,-20.050000,-52.500,99.000,106.900,...,24,0,0,0,14,0,9,1580,24,0
3,2022-02-07,1032,control,151,4.273468,3.418774,-73.071429,-79.500,-26.099,-15.700,...,117,0,0,0,61,36,12,5565,117,36
4,2022-02-07,1052,variant1,361,4.065008,6.504012,-58.831776,-60.000,-18.900,-8.199,...,123,0,0,0,73,39,10,5862,123,39


In [104]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   date                                          4032 non-null   object
 1   zone_id                                       4032 non-null   int16 
 2   variant                                       4032 non-null   object
 3   region_id                                     4032 non-null   int16 
 4   planned_late_percent                          4032 non-null   object
 5   late_percent                                  4032 non-null   object
 6   avg_lateness_minutes                          4032 non-null   object
 7   median_lateness_minutes                       4032 non-null   object
 8   p85_lateness_minutes                          4032 non-null   object
 9   p90_lateness_minutes                          4032 non-null   object
 10  

In [105]:
# Create a dataframe from the noncost_query defined above
df4=iq.query(noncost_query_2)
df4.head()

,date,zone_id,lost_demand_pct,capacity_lost_demand_pct,total_demand
0,2022-02-25,441,0.014568,0.004762,1671.348148
1,2022-02-16,2234,0.041894,0.008054,5.218628
2,2022-02-13,711,0.047845,0.014956,211.100166
3,2022-02-07,837,0.027000,0.009651,92.497405
4,2022-02-25,828,0.014565,0.006513,544.937280


In [106]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   date                      4032 non-null   object 
 1   zone_id                   4032 non-null   int16  
 2   lost_demand_pct           4032 non-null   float64
 3   capacity_lost_demand_pct  4032 non-null   float64
 4   total_demand              4032 non-null   float64
dtypes: float64(3), int16(1), object(1)
memory usage: 134.0+ KB


In [107]:
df5=pd.merge(df3,df4, on=['date','zone_id'])
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4032 entries, 0 to 4031
Data columns (total 77 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   date                                          4032 non-null   object 
 1   zone_id                                       4032 non-null   int16  
 2   variant                                       4032 non-null   object 
 3   region_id                                     4032 non-null   int16  
 4   planned_late_percent                          4032 non-null   object 
 5   late_percent                                  4032 non-null   object 
 6   avg_lateness_minutes                          4032 non-null   object 
 7   median_lateness_minutes                       4032 non-null   object 
 8   p85_lateness_minutes                          4032 non-null   object 
 9   p90_lateness_minutes                          4032 non-null   o

In [108]:
# Create a list of column names and remove the first 4 columns from the list
lists=(df5.columns).to_list()
del lists[0:4]

In [110]:
# Convert the columns in the above list to numeric data
df5
for i in lists: 
    df5[i]=pd.to_numeric(df5[i])

In [111]:
# Change the date column to datetime datatype
df5['date']=pd.to_datetime(df5['date'])
# Create a day of week column
df5['dow']=[i.weekday() for i in df5['date']]
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4032 entries, 0 to 4031
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   date                                          4032 non-null   datetime64[ns]
 1   zone_id                                       4032 non-null   int16         
 2   variant                                       4032 non-null   object        
 3   region_id                                     4032 non-null   int16         
 4   planned_late_percent                          4032 non-null   float64       
 5   late_percent                                  4032 non-null   float64       
 6   avg_lateness_minutes                          4032 non-null   float64       
 7   median_lateness_minutes                       4032 non-null   float64       
 8   p85_lateness_minutes                          4032 non-null   float6

In [112]:
df5.head()

,date,zone_id,variant,region_id,planned_late_percent,late_percent,avg_lateness_minutes,median_lateness_minutes,p85_lateness_minutes,p90_lateness_minutes,...,standard_eta_deliveries_count,priority_eta_deliveries_count,scheduled_deliveries_count,active_minutes_full_service,deliveries,deliveries_priority_eta,lost_demand_pct,capacity_lost_demand_pct,total_demand,dow
0,2022-02-07,1920,variant1,191,3.658492,13.414471,-48.506329,-55.0,-4.400,5.800,...,50,27,5,3974.0,82,27,0.023530,0.008237,83.975932,0
1,2022-02-07,2234,control,1,33.327779,49.991668,-39.166667,-24.5,35.500,49.000,...,0,0,2,438.0,6,0,0.054719,0.050310,6.347318,0
2,2022-02-07,1925,variant1,143,24.998958,20.832465,-20.050000,-52.5,99.000,106.900,...,14,0,9,1580.0,24,0,0.033624,0.006970,23.800251,0
3,2022-02-07,1032,control,151,4.273468,3.418774,-73.071429,-79.5,-26.099,-15.700,...,61,36,12,5565.0,117,36,0.029726,0.010261,115.431291,0
4,2022-02-07,1052,variant1,361,4.065008,6.504012,-58.831776,-60.0,-18.900,-8.199,...,73,39,10,5862.0,123,39,0.017900,0.008740,131.351132,0


In [113]:
# Create a dataframe from the noncost_query defined above
df6=iq.query(noncost_query_3)
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     4032 non-null   object
 1   zone_id  4032 non-null   int16 
 2   exp_590  4032 non-null   object
 3   exp_581  4032 non-null   object
 4   exp_607  4032 non-null   object
dtypes: int16(1), object(4)
memory usage: 134.0+ KB


In [114]:
# Change the date column to datetime datatype
df6['date']=pd.to_datetime(df6['date'])
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     4032 non-null   datetime64[ns]
 1   zone_id  4032 non-null   int16         
 2   exp_590  4032 non-null   object        
 3   exp_581  4032 non-null   object        
 4   exp_607  4032 non-null   object        
dtypes: datetime64[ns](1), int16(1), object(3)
memory usage: 134.0+ KB


In [115]:
# Create a list of column names and remove the first 4 columns from the list
lists=(df6.columns).to_list()
del lists[0:2]

In [116]:
# Convert the columns in the above list to numeric data
for i in lists: 
    df6[i]=pd.to_numeric(df6[i])

In [117]:
df7=pd.merge(df5,df6, on=['date','zone_id'])
df7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4032 entries, 0 to 4031
Data columns (total 81 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   date                                          4032 non-null   datetime64[ns]
 1   zone_id                                       4032 non-null   int16         
 2   variant                                       4032 non-null   object        
 3   region_id                                     4032 non-null   int16         
 4   planned_late_percent                          4032 non-null   float64       
 5   late_percent                                  4032 non-null   float64       
 6   avg_lateness_minutes                          4032 non-null   float64       
 7   median_lateness_minutes                       4032 non-null   float64       
 8   p85_lateness_minutes                          4032 non-null   float6

In [118]:
# Create a dataframe from the noncost_query defined above
df8=iq.query(noncost_query_4)
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   zone_id                 4032 non-null   int16  
 1   date                    4032 non-null   object 
 2   variant                 4032 non-null   object 
 3   guaranteed_pay_per_del  4032 non-null   float64
dtypes: float64(1), int16(1), object(2)
memory usage: 102.5+ KB


In [119]:
# Change the date column to datetime datatype
df8['date']=pd.to_datetime(df8['date'])
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   zone_id                 4032 non-null   int16         
 1   date                    4032 non-null   datetime64[ns]
 2   variant                 4032 non-null   object        
 3   guaranteed_pay_per_del  4032 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int16(1), object(1)
memory usage: 102.5+ KB


In [127]:
df9=pd.merge(df7,df8, on=['date','zone_id','variant'])
df9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4032 entries, 0 to 4031
Data columns (total 82 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   date                                          4032 non-null   datetime64[ns]
 1   zone_id                                       4032 non-null   int16         
 2   variant                                       4032 non-null   object        
 3   region_id                                     4032 non-null   int16         
 4   planned_late_percent                          4032 non-null   float64       
 5   late_percent                                  4032 non-null   float64       
 6   avg_lateness_minutes                          4032 non-null   float64       
 7   median_lateness_minutes                       4032 non-null   float64       
 8   p85_lateness_minutes                          4032 non-null   float6

In [128]:
# Create region-day clusters 
df9['region_day']=df9.groupby(['region_id', 'date']).ngroup()
df9['region_day']=pd.to_numeric(df9['region_day'])

In [130]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4032 entries, 0 to 4031
Data columns (total 83 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   date                                          4032 non-null   datetime64[ns]
 1   zone_id                                       4032 non-null   int16         
 2   variant                                       4032 non-null   object        
 3   region_id                                     4032 non-null   int16         
 4   planned_late_percent                          4032 non-null   float64       
 5   late_percent                                  4032 non-null   float64       
 6   avg_lateness_minutes                          4032 non-null   float64       
 7   median_lateness_minutes                       4032 non-null   float64       
 8   p85_lateness_minutes                          4032 non-null   float6

## Variant 1 Results

In [140]:
# Divide the data into control and treatment for 
control_data=df9[df9['variant']=='control']
v1_data=df9[df9['variant']=='variant1']
v2_data=df9[df9['variant']=='variant2']

In [132]:
# Create two dataframes where each dataframe contains the control and one of the treatments 
df_v1=df9[(df9['variant']=='control')|(df9['variant']=='variant1')]
df_v2=df9[(df9['variant']=='control')|(df9['variant']=='variant2')]

In [134]:
# Calculate the coefficient and confidence interval for several metrics
metrics= {
  'planned_late_percent':'deliveries_count'
    , 'late_percent':'deliveries_count'
      , 'avg_lateness_minutes':'deliveries_count'
       , 'median_lateness_minutes':'deliveries_count'
        , 'p85_lateness_minutes':'deliveries_count'
        , 'p90_lateness_minutes':'deliveries_count'
        , 'p95_lateness_minutes':'deliveries_count'
       , 'p99_lateness_minutes':'deliveries_count'
        ,'scheduled_early_percent':'scheduled_deliveries_count'
     ,'pickup_late_percent':'pickup_deliveries_count'
    , 'delivery_late_percent':'delivery_deliveries_count'
        ,'pickup_planned_late_percent':'pickup_deliveries_count'
        
      ,  'delivery_planned_late_percent':'delivery_deliveries_count'
        ,  'standard_eta_late_percent':'standard_eta_deliveries_count'
        ,  'priority_eta_late_percent':'priority_eta_deliveries_count'  
      ,  'scheduled_late_percent':'scheduled_deliveries_count'
        , 'delivery_15_minute_late_percent':'delivery_deliveries_count' 
            
     , 'standard_eta_15_minute_late_percent':'standard_eta_deliveries_count'
        , 'priority_eta_15_minute_late_percent':'priority_eta_deliveries_count'
        , 'scheduled_15_minute_late_percent':'scheduled_deliveries_count'
       , 'delivery_30_minute_late_percent':'delivery_deliveries_count'
       , 'standard_eta_30_minute_late_percent':'standard_eta_deliveries_count'                 
      , 'priority_eta_30_minute_late_percent':'priority_eta_deliveries_count'
       
     ,'scheduled_30_minute_late_percent':'scheduled_deliveries_count' 
     ,'average_fulfillment_minutes_standard_eta':'standard_eta_deliveries_count'     
    , 'average_fulfillment_minutes_priority_eta':'priority_eta_deliveries_count'
    ,'fs_average_hourly_rate':'active_minutes_full_service'
    ,'total_cpd':'deliveries'
    ,'fs_cpd':'deliveries'
    ,'total_priority_eta_cpd':'deliveries_priority_eta'
    , 'fs_priority_eta_cpd':'deliveries_priority_eta' 
      , 'driving_time_cpd':'deliveries'
      , 'driving_mileage_cpd':'deliveries'
      , 'picking_time_cpd':'deliveries'
      , 'delivery_time_cpd':'deliveries'
      , 'mileage_cpd':'deliveries'   
      , 'batch_payment_floor_cpd':'deliveries'
      , 'batch_incentive_floor_cpd':'deliveries'
      , 'heavy_item_bump_cpd':'deliveries'
      , 'on_demand_acceptance_boost_cpd':'deliveries'
      , 'on_demand_acceptance_boost_priority_eta_cpd':'deliveries_priority_eta'
      , 'add_on_batch_floor_cpd':'deliveries_priority_eta'
      , 'hourly_base_cpd':'deliveries'
      , 'service_bonus_cpd':'deliveries'
      , 'ml_adjustment_cpd':'deliveries'
      , 'time_adjustment_cpd':'deliveries'
      , 'random_adjustment_cpd':'deliveries'
      , 'hourly_bonus_cpd':'deliveries'
       , 'quest_bonus_cpd':'deliveries' 
       , 'tip_per_del':'deliveries'
       , 'experiment_cost_adjustment_cpd':'deliveries'
       , 'peak_time_pay_cpd':'deliveries'
       , 'experiment_cost_adjustment_priority_eta_cpd':'deliveries_priority_eta'
    , 'avg_time_to_accept_per_delivery':'deliveries'
     , 'avg_time_to_accept_per_delivery_priority_eta':'deliveries_priority_eta'
     , 'tta_percentile_25': 'deliveries'
     , 'tta_percentile_75': 'deliveries'
     , 'tta_percentile_90': 'deliveries'
     , 'tta_median': 'deliveries'
     , 'lost_demand_pct':'total_demand'                              
     , 'capacity_lost_demand_pct':'total_demand'  
     , 'guaranteed_pay_per_del':'deliveries'    
}



results=pd.DataFrame(index=metrics.keys(), columns=['control_mean','estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric, weight in metrics.items():
    equation = '%s ~ variant + C(zone_id) + C(dow)+C(exp_590)+C(exp_581)+C(exp_607)'
    model = smf.wls( equation % metric, data=df_v1, weights=df_v1[weight]).fit()
    #Using robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=df_v1['region_day'])
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['control_mean']=DescrStatsW(control_data[metric], weights=control_data[weight]).mean
    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

,control_mean,estimate,p-value,[0.025,0.975],significance
planned_late_percent,5.892195,-0.314426,0.032689,-0.602893,-0.02596,significant
late_percent,7.921814,-0.110553,0.575773,-0.498187,0.27708,not significant
avg_lateness_minutes,-48.551978,-0.349408,0.431401,-1.220645,0.521828,not significant
median_lateness_minutes,-44.533539,-0.444984,0.377535,-1.434222,0.544254,not significant
p85_lateness_minutes,-11.101535,-0.323293,0.460514,-1.182732,0.536145,not significant
p90_lateness_minutes,-4.167314,-0.307536,0.488487,-1.178545,0.563473,not significant
p95_lateness_minutes,7.907084,-0.37308,0.440656,-1.322304,0.576143,not significant
p99_lateness_minutes,43.693841,-1.254127,0.142915,-2.932795,0.42454,not significant
scheduled_early_percent,27.327357,0.203499,0.582245,-0.522306,0.929304,not significant
pickup_late_percent,13.559394,0.57218,0.333181,-0.587676,1.732037,not significant


In [135]:
results_significant=results[results['significance']=='significant']
results_significant

,control_mean,estimate,p-value,[0.025,0.975],significance
planned_late_percent,5.892195,-0.314426,0.032689,-0.602893,-0.02596,significant
delivery_planned_late_percent,5.75075,-0.330247,0.027419,-0.623638,-0.036857,significant
fs_average_hourly_rate,13.627638,0.834642,0.0,0.775746,0.893538,significant
total_cpd,10.032669,0.612668,0.0,0.566113,0.659222,significant
fs_cpd,9.878007,0.612052,0.0,0.566797,0.657307,significant
total_priority_eta_cpd,10.318487,0.555726,0.0,0.5074,0.604051,significant
fs_priority_eta_cpd,10.318487,0.555726,0.0,0.5074,0.604051,significant
driving_time_cpd,1.127168,0.065419,0.0,0.047523,0.083315,significant
picking_time_cpd,4.262777,0.340857,0.0,0.302419,0.379296,significant
delivery_time_cpd,2.816828,0.209287,0.0,0.197019,0.221556,significant


In [136]:
results_not_significant=results[results['significance']=='not significant']
results_not_significant

,control_mean,estimate,p-value,[0.025,0.975],significance
late_percent,7.921814,-0.110553,0.575773,-0.498187,0.27708,not significant
avg_lateness_minutes,-48.551978,-0.349408,0.431401,-1.220645,0.521828,not significant
median_lateness_minutes,-44.533539,-0.444984,0.377535,-1.434222,0.544254,not significant
p85_lateness_minutes,-11.101535,-0.323293,0.460514,-1.182732,0.536145,not significant
p90_lateness_minutes,-4.167314,-0.307536,0.488487,-1.178545,0.563473,not significant
p95_lateness_minutes,7.907084,-0.37308,0.440656,-1.322304,0.576143,not significant
p99_lateness_minutes,43.693841,-1.254127,0.142915,-2.932795,0.42454,not significant
scheduled_early_percent,27.327357,0.203499,0.582245,-0.522306,0.929304,not significant
pickup_late_percent,13.559394,0.57218,0.333181,-0.587676,1.732037,not significant
delivery_late_percent,7.828833,-0.122846,0.542125,-0.518223,0.272531,not significant


## Variant 2 Results

In [137]:
# Calculate the coefficient and confidence interval for several metrics
metrics= {
  'planned_late_percent':'deliveries_count'
    , 'late_percent':'deliveries_count'
      , 'avg_lateness_minutes':'deliveries_count'
       , 'median_lateness_minutes':'deliveries_count'
        , 'p85_lateness_minutes':'deliveries_count'
        , 'p90_lateness_minutes':'deliveries_count'
        , 'p95_lateness_minutes':'deliveries_count'
       , 'p99_lateness_minutes':'deliveries_count'
        ,'scheduled_early_percent':'scheduled_deliveries_count'
     ,'pickup_late_percent':'pickup_deliveries_count'
    , 'delivery_late_percent':'delivery_deliveries_count'
        ,'pickup_planned_late_percent':'pickup_deliveries_count'
        
      ,  'delivery_planned_late_percent':'delivery_deliveries_count'
        ,  'standard_eta_late_percent':'standard_eta_deliveries_count'
        ,  'priority_eta_late_percent':'priority_eta_deliveries_count'  
      ,  'scheduled_late_percent':'scheduled_deliveries_count'
        , 'delivery_15_minute_late_percent':'delivery_deliveries_count' 
            
     , 'standard_eta_15_minute_late_percent':'standard_eta_deliveries_count'
        , 'priority_eta_15_minute_late_percent':'priority_eta_deliveries_count'
        , 'scheduled_15_minute_late_percent':'scheduled_deliveries_count'
       , 'delivery_30_minute_late_percent':'delivery_deliveries_count'
       , 'standard_eta_30_minute_late_percent':'standard_eta_deliveries_count'                 
      , 'priority_eta_30_minute_late_percent':'priority_eta_deliveries_count'
       
     ,'scheduled_30_minute_late_percent':'scheduled_deliveries_count' 
     ,'average_fulfillment_minutes_standard_eta':'standard_eta_deliveries_count'     
    , 'average_fulfillment_minutes_priority_eta':'priority_eta_deliveries_count'
    ,'fs_average_hourly_rate':'active_minutes_full_service'
    ,'total_cpd':'deliveries'
    ,'fs_cpd':'deliveries'
    ,'total_priority_eta_cpd':'deliveries_priority_eta'
    , 'fs_priority_eta_cpd':'deliveries_priority_eta' 
      , 'driving_time_cpd':'deliveries'
      , 'driving_mileage_cpd':'deliveries'
      , 'picking_time_cpd':'deliveries'
      , 'delivery_time_cpd':'deliveries'
      , 'mileage_cpd':'deliveries'   
      , 'batch_payment_floor_cpd':'deliveries'
      , 'batch_incentive_floor_cpd':'deliveries'
      , 'heavy_item_bump_cpd':'deliveries'
      , 'on_demand_acceptance_boost_cpd':'deliveries'
      , 'on_demand_acceptance_boost_priority_eta_cpd':'deliveries_priority_eta'
      , 'add_on_batch_floor_cpd':'deliveries_priority_eta'
      , 'hourly_base_cpd':'deliveries'
      , 'service_bonus_cpd':'deliveries'
      , 'ml_adjustment_cpd':'deliveries'
      , 'time_adjustment_cpd':'deliveries'
      , 'random_adjustment_cpd':'deliveries'
      , 'hourly_bonus_cpd':'deliveries'
       , 'quest_bonus_cpd':'deliveries' 
       , 'tip_per_del':'deliveries'
       , 'experiment_cost_adjustment_cpd':'deliveries'
       , 'peak_time_pay_cpd':'deliveries'
       , 'experiment_cost_adjustment_priority_eta_cpd':'deliveries_priority_eta'
    , 'avg_time_to_accept_per_delivery':'deliveries'
     , 'avg_time_to_accept_per_delivery_priority_eta':'deliveries_priority_eta'
     , 'tta_percentile_25': 'deliveries'
     , 'tta_percentile_75': 'deliveries'
     , 'tta_percentile_90': 'deliveries'
     , 'tta_median': 'deliveries'
     , 'lost_demand_pct':'total_demand'                              
     , 'capacity_lost_demand_pct':'total_demand'  
     , 'guaranteed_pay_per_del':'deliveries'    
}



results=pd.DataFrame(index=metrics.keys(), columns=['control_mean','estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric, weight in metrics.items():
    equation = '%s ~ variant + C(zone_id) + C(dow)+C(exp_590)+C(exp_581)+C(exp_607)'
    model = smf.wls( equation % metric, data=df_v2, weights=df_v2[weight]).fit()
    #Using robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=df_v2['region_day'])
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['control_mean']=DescrStatsW(control_data[metric], weights=control_data[weight]).mean
    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

,control_mean,estimate,p-value,[0.025,0.975],significance
planned_late_percent,5.892195,0.329693,0.022384,0.046863,0.612523,significant
late_percent,7.921814,0.311062,0.09309,-0.052095,0.674219,not significant
avg_lateness_minutes,-48.551978,0.640451,0.147128,-0.225844,1.506745,not significant
median_lateness_minutes,-44.533539,0.514873,0.321112,-0.503086,1.532833,not significant
p85_lateness_minutes,-11.101535,0.496519,0.226967,-0.309509,1.302547,not significant
p90_lateness_minutes,-4.167314,0.69995,0.097969,-0.12936,1.52926,not significant
p95_lateness_minutes,7.907084,1.074075,0.022557,0.151486,1.996663,significant
p99_lateness_minutes,43.693841,2.389582,0.000994,0.969987,3.809177,significant
scheduled_early_percent,27.327357,-0.859188,0.017888,-1.569971,-0.148405,significant
pickup_late_percent,13.559394,1.029101,0.060028,-0.043536,2.101738,not significant


In [138]:
results_significant=results[results['significance']=='significant']
results_significant

,control_mean,estimate,p-value,[0.025,0.975],significance
planned_late_percent,5.892195,0.329693,0.022384,0.046863,0.612523,significant
p95_lateness_minutes,7.907084,1.074075,0.022557,0.151486,1.996663,significant
p99_lateness_minutes,43.693841,2.389582,0.000994,0.969987,3.809177,significant
scheduled_early_percent,27.327357,-0.859188,0.017888,-1.569971,-0.148405,significant
delivery_planned_late_percent,5.75075,0.317603,0.0294,0.031883,0.603323,significant
standard_eta_late_percent,6.342696,0.515094,0.001246,0.202974,0.827214,significant
scheduled_late_percent,2.781556,0.248113,0.042407,0.008521,0.487704,significant
delivery_15_minute_late_percent,3.403444,0.229627,0.006978,0.062978,0.396276,significant
standard_eta_15_minute_late_percent,3.302808,0.339261,0.000283,0.156634,0.521887,significant
scheduled_15_minute_late_percent,1.658548,0.17123,0.034041,0.012926,0.329535,significant


In [139]:
results_not_significant=results[results['significance']=='not significant']
results_not_significant

,control_mean,estimate,p-value,[0.025,0.975],significance
late_percent,7.921814,0.311062,0.09309,-0.052095,0.674219,not significant
avg_lateness_minutes,-48.551978,0.640451,0.147128,-0.225844,1.506745,not significant
median_lateness_minutes,-44.533539,0.514873,0.321112,-0.503086,1.532833,not significant
p85_lateness_minutes,-11.101535,0.496519,0.226967,-0.309509,1.302547,not significant
p90_lateness_minutes,-4.167314,0.69995,0.097969,-0.12936,1.52926,not significant
pickup_late_percent,13.559394,1.029101,0.060028,-0.043536,2.101738,not significant
delivery_late_percent,7.828833,0.298319,0.110783,-0.068475,0.665114,not significant
pickup_planned_late_percent,14.46837,1.026493,0.05992,-0.042972,2.095958,not significant
priority_eta_late_percent,13.111962,0.02278,0.950884,-0.702889,0.748449,not significant
priority_eta_15_minute_late_percent,4.470826,0.0723,0.564756,-0.174064,0.318664,not significant


In [141]:
v2_data['zone_id'].unique()

array([ 779,  618,  785, 1473, 1075,  571,  616, 1057, 1077,  620,  443,
       1078,  436, 1910, 1002,  441,  615,  438,  283,  983,  559,  613,
        872, 1049,  611,  839,  439, 1053,  442, 1076, 1474, 1916, 1001,
        572, 1826,  440,  617,  972, 1073,  437, 1050, 1822,  619,  832,
       1921, 1918, 1922,  561, 1054,  562, 1028, 1915,  574,  828, 1920,
       1914,  696, 1052, 1925, 1919, 1051, 1056, 1917,  541,  701,  836,
        603,  837,  550,  602, 1032,  942, 2234, 1430,    1,  162,  400,
        979,  980, 1590,  523,  711,  698,  521,  524,  981,  522,  161,
        697,  858,  160,   95, 1055, 1058, 1440,  859], dtype=int16)

In [142]:
j=0
zones=pd.DataFrame(columns=['zone_id','cld','tta_percentile_90','supply'], index=range(0,96), data=np.zeros((96,4)))
for i in v2_data['zone_id'].unique():
    zones['zone_id'].iloc[j]=i
    zones['cld'].iloc[j]=np.average(a=v2_data['capacity_lost_demand_pct'][v2_data['zone_id']==i], weights=v2_data['total_demand'][v2_data['zone_id']==i])
    zones['tta_percentile_90'].iloc[j]=np.average(a=v2_data['tta_percentile_90'][v2_data['zone_id']==i], weights=v2_data['deliveries'][v2_data['zone_id']==i])
      
    if zones['cld'].iloc[j]>0.025:
        zones['supply'].iloc[j]='undersupplied'
    if  (zones['cld'].iloc[j]<0.0125) & (zones['tta_percentile_90'].iloc[j]<25):
        zones['supply'].iloc[j]='oversupplied'  
    else: 
        zones['supply'].iloc[j]='balanced'  
    j=j+1
  

In [144]:
zones

,zone_id,cld,tta_percentile_90,supply
0,779.0,0.007123,17.888492,oversupplied
1,618.0,0.004298,5.362953,oversupplied
2,785.0,0.005821,12.467767,oversupplied
3,1473.0,0.007717,20.781196,oversupplied
4,1075.0,0.006094,11.276155,oversupplied
...,...,...,...,...
91,95.0,0.006727,22.637241,oversupplied
92,1055.0,0.009104,24.232891,oversupplied
93,1058.0,0.024764,43.049898,balanced
94,1440.0,0.028302,193.004807,balanced


In [ ]:
# Calculate the coefficient and confidence interval for several metrics
metrics= {
  'planned_late_percent':'deliveries_count'
    , 'late_percent':'deliveries_count'
      , 'avg_lateness_minutes':'deliveries_count'
       , 'median_lateness_minutes':'deliveries_count'
        , 'p85_lateness_minutes':'deliveries_count'
        , 'p90_lateness_minutes':'deliveries_count'
        , 'p95_lateness_minutes':'deliveries_count'
       , 'p99_lateness_minutes':'deliveries_count'
        ,'scheduled_early_percent':'scheduled_deliveries_count'
     ,'pickup_late_percent':'pickup_deliveries_count'
    , 'delivery_late_percent':'delivery_deliveries_count'
        ,'pickup_planned_late_percent':'pickup_deliveries_count'
        
      ,  'delivery_planned_late_percent':'delivery_deliveries_count'
        ,  'standard_eta_late_percent':'standard_eta_deliveries_count'
        ,  'priority_eta_late_percent':'priority_eta_deliveries_count'  
      ,  'scheduled_late_percent':'scheduled_deliveries_count'
        , 'delivery_15_minute_late_percent':'delivery_deliveries_count' 
            
     , 'standard_eta_15_minute_late_percent':'standard_eta_deliveries_count'
        , 'priority_eta_15_minute_late_percent':'priority_eta_deliveries_count'
        , 'scheduled_15_minute_late_percent':'scheduled_deliveries_count'
       , 'delivery_30_minute_late_percent':'delivery_deliveries_count'
       , 'standard_eta_30_minute_late_percent':'standard_eta_deliveries_count'                 
      , 'priority_eta_30_minute_late_percent':'priority_eta_deliveries_count'
       
     ,'scheduled_30_minute_late_percent':'scheduled_deliveries_count' 
     ,'average_fulfillment_minutes_standard_eta':'standard_eta_deliveries_count'     
    , 'average_fulfillment_minutes_priority_eta':'priority_eta_deliveries_count'
    ,'fs_average_hourly_rate':'active_minutes_full_service'
    ,'total_cpd':'deliveries'
    ,'fs_cpd':'deliveries'
    ,'total_priority_eta_cpd':'deliveries_priority_eta'
    , 'fs_priority_eta_cpd':'deliveries_priority_eta' 
      , 'driving_time_cpd':'deliveries'
      , 'driving_mileage_cpd':'deliveries'
      , 'picking_time_cpd':'deliveries'
      , 'delivery_time_cpd':'deliveries'
      , 'mileage_cpd':'deliveries'   
      , 'batch_payment_floor_cpd':'deliveries'
      , 'batch_incentive_floor_cpd':'deliveries'
      , 'heavy_item_bump_cpd':'deliveries'
      , 'on_demand_acceptance_boost_cpd':'deliveries'
      , 'on_demand_acceptance_boost_priority_eta_cpd':'deliveries_priority_eta'
      , 'add_on_batch_floor_cpd':'deliveries_priority_eta'
      , 'hourly_base_cpd':'deliveries'
      , 'service_bonus_cpd':'deliveries'
      , 'ml_adjustment_cpd':'deliveries'
      , 'time_adjustment_cpd':'deliveries'
      , 'random_adjustment_cpd':'deliveries'
      , 'hourly_bonus_cpd':'deliveries'
       , 'quest_bonus_cpd':'deliveries' 
       , 'tip_per_del':'deliveries'
       , 'experiment_cost_adjustment_cpd':'deliveries'
       , 'peak_time_pay_cpd':'deliveries'
       , 'experiment_cost_adjustment_priority_eta_cpd':'deliveries_priority_eta'
    , 'avg_time_to_accept_per_delivery':'deliveries'
     , 'avg_time_to_accept_per_delivery_priority_eta':'deliveries_priority_eta'
     , 'tta_percentile_25': 'deliveries'
     , 'tta_percentile_75': 'deliveries'
     , 'tta_percentile_90': 'deliveries'
     , 'tta_median': 'deliveries'
     , 'lost_demand_pct':'total_demand'                              
     , 'capacity_lost_demand_pct':'total_demand'  
     , 'guaranteed_pay_per_del':'deliveries'    
}



results=pd.DataFrame(index=metrics.keys(), columns=['control_mean','estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric, weight in metrics.items():
    equation = '%s ~ variant + C(zone_id) + C(dow)+C(exp_590)+C(exp_581)+C(exp_607)'
    model = smf.wls( equation % metric, data=df, weights=df[weight]).fit()
    #Using robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=df['region_id'])
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['control_mean']=DescrStatsW(control_data[metric], weights=control_data[weight]).mean
    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

## Variant 2 Results

In [ ]:
# If the variant 2 treatment data shows that the zone is still non-undersupplied, then we decrease the discount factor          

Generally >2.5% CLD for undersupply

<1.25% CLD and <25min 90th percentile TTA for oversupply




In [ ]:
results_important=results[results.index.isin(['TTA','LATES','ACCEPTANCE_BOOST_COST_PER_DELIVERY', 'HEAVY_PAY_PER_DELIVERY',
       'MILEAGE_PER_DELIVERY', 'BATCH_PAYMENT_FLOOR_PER_DELIVERY',
       'PICKING_PAY_PER_DELIVERY',
       'FS_LABOR_COST_WITHOUT_PICKING_PER_DELIVERY',
       'FS_LABOR_COST_PER_DELIVERY','TWO_HR_CAPACITY_AVAILABILITY','LOST_DEMAND'])]
results_important

In [ ]:
df_small=df[df['ZONE_SIZE'].isin(['0 - 200','201 - 500'])]
df_big=df[~df['ZONE_SIZE'].isin(['0 - 200','201 - 500'])]

In [ ]:
# Calculate the coefficient and confidence interval for several metrics
metrics= ['LATES', 'TTA',
       'TWO_HOUR_DELIVERY', 'FIVE_HOUR_DELIVERY', 'TWO_HOUR_TOTAL_VISITS',
       'TWO_HOUR_AVAILABILITY', 'TWO_HOUR_CAPACITY_OFF_UNAVAILABILITY',
       'TWO_HOUR_TIME_MODEL_UNAVAILABILITY', 'NO_2HR_OPTION_UNAVAILABILIY',
       'TWO_HOUR_CUSTOMER_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_BLOCK_EVENTS_UNAVAILABILITY',
       'TWO_HOUR_ALCOHOL_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_ESO_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_OPEN_DELAY_UNAVAILABILITY', 'TWO_HOUR_UNKNOWN_UNAVAILABILITY',
       'TWO_HR_CAPACITY_AVAILABILITY', 'FIVE_HOUR_TOTAL_VISITS',
       'FIVE_HOUR_AVAILABILITY', 'FIVE_HOUR_CAPACITY_OFF_UNAVAILABILITY',
       'FIVE_HOUR_TIME_MODEL_UNAVAILABILITY', 'NO_5HR_OPTION_UNAVAILABILIY',
       'FIVE_HOUR_CUSTOMER_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_BLOCK_EVENTS_UNAVAILABILITY',
       'FIVE_HOUR_ALCOHOL_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_ESO_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_OPEN_DELAY_UNAVAILABILITY',
       'FIVE_HOUR_UNKNOWN_UNAVAILABILITY', 'FIVE_HR_CAPACITY_AVAILABILITY',
       'ACCEPTANCE_BOOST_COST_PER_DELIVERY', 'DELIVERIES',
       'DELIVERIES_PER_ONLINE_SHOPPER', 'LOST_DEMAND', 'TOTAL_LOST_DELIVERIES']
results=pd.DataFrame(index=metrics, columns=['estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric in metrics:
    equation = '%s ~ VARIANT + C(ZONE_ID) + C(DOW)'
    model = smf.wls( equation % metric, data=df_small, weights=df_small['DELIVERIES']).fit()
    #Using region-clustered robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='cluster', groups=cost_df['region_id'].T)    
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

In [ ]:
# Calculate the coefficient and confidence interval for several metrics
metrics= ['LATES', 'TTA',
       'TWO_HOUR_DELIVERY', 'FIVE_HOUR_DELIVERY', 'TWO_HOUR_TOTAL_VISITS',
       'TWO_HOUR_AVAILABILITY', 'TWO_HOUR_CAPACITY_OFF_UNAVAILABILITY',
       'TWO_HOUR_TIME_MODEL_UNAVAILABILITY', 'NO_2HR_OPTION_UNAVAILABILIY',
       'TWO_HOUR_CUSTOMER_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_BLOCK_EVENTS_UNAVAILABILITY',
       'TWO_HOUR_ALCOHOL_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_ESO_FORCE_OFF_UNAVAILABILITY',
       'TWO_HOUR_OPEN_DELAY_UNAVAILABILITY', 'TWO_HOUR_UNKNOWN_UNAVAILABILITY',
       'TWO_HR_CAPACITY_AVAILABILITY', 'FIVE_HOUR_TOTAL_VISITS',
       'FIVE_HOUR_AVAILABILITY', 'FIVE_HOUR_CAPACITY_OFF_UNAVAILABILITY',
       'FIVE_HOUR_TIME_MODEL_UNAVAILABILITY', 'NO_5HR_OPTION_UNAVAILABILIY',
       'FIVE_HOUR_CUSTOMER_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_BLOCK_EVENTS_UNAVAILABILITY',
       'FIVE_HOUR_ALCOHOL_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_ESO_FORCE_OFF_UNAVAILABILITY',
       'FIVE_HOUR_OPEN_DELAY_UNAVAILABILITY',
       'FIVE_HOUR_UNKNOWN_UNAVAILABILITY', 'FIVE_HR_CAPACITY_AVAILABILITY',
       'ACCEPTANCE_BOOST_COST_PER_DELIVERY', 'DELIVERIES',
       'DELIVERIES_PER_ONLINE_SHOPPER', 'LOST_DEMAND', 'TOTAL_LOST_DELIVERIES']
results=pd.DataFrame(index=metrics, columns=['estimate', 'p-value','[0.025', '0.975]', 'significance'])
for metric in metrics:
    equation = '%s ~ VARIANT + C(ZONE_ID) + C(DOW)'
    model = smf.wls( equation % metric, data=df_big, weights=df_big['DELIVERIES']).fit()
    #Using HC robust standard errors as below
    robust_fit = model.get_robustcov_results(cov_type='HC1')
    params = pd.Series(robust_fit.params, index=robust_fit.model.exog_names)
    pvalues = pd.Series(robust_fit.pvalues, index=robust_fit.model.exog_names)

    results.loc[metric]['estimate']=params[1]
    results.loc[metric]['p-value']=pvalues[1]
    results.loc[metric]['[0.025']=robust_fit.conf_int()[1][0]
    results.loc[metric]['0.975]']=robust_fit.conf_int()[1][1]
    if ((results.loc[metric]['[0.025']>0) & (results.loc[metric]['0.975]']>0)) | ((results.loc[metric]['[0.025']<0) & (results.loc[metric]['0.975]']<0)):
        results.loc[metric]['significance']='significant'
    else: 
        results.loc[metric]['significance']='not significant'
        
results

In [ ]:
# Show the summary results for a metric
robust_fit.summary()